<a name="cell-solving"></a>

### 3. [$Ax=b$](#first_cell) [and Singular Values](#first_cell)

0. [Linearly Transforming](#cell-transforming-Axb) [$x$](#cell-transforming-Axb) [with](#cell-transforming-Axb) [$Ax=b$](#cell-transforming-Axb)
  - [[PREREQUESITE] Linear Independence, Orthogonality, Rank, and Bases](#cell-Axb-etc): Week 2 Programming Assignment Problem 1
  - [[PREREQUESITE] Eigenvalues and Eigenvectors](#cell-sovling-Axb-math-eig2)
  - [[PREREQUESITE] Eigendecomposition](#cell-sovling-Axb-math-eig1)
  - [Understanding](#cell-sovling-Axb-Eigenanalysis) [$Ax$](#cell-sovling-Axb-Eigenanalysis) [by its Eigenvalues](#cell-sovling-Axb-Eigenanalysis)

1. [Singular Value Decomposition (SVD)](#cell-sovling-Axb-math-svd)
    0. [Understanding](#cell-sovling-Axb-UDVt) [$Ax$](#cell-sovling-Axb-UDVt) [with](#cell-sovling-Axb-UDVt) [$Ax=UDV^Tx$](#cell-sovling-Axb-UDVt)
    1. [Multicollinearity and Variance Inflation Factors (VIFs)](#cell-sovling-Axb-math-multicollinearity)
    2. [Principal Components Analysis (PCA)](#cell-sovling-Axb-math-pca)
    3. [Understanding PCA as](#cell-sovling-X-UDVt) [$X=UDV^T$](#cell-sovling-X-UDVt)
    4. [Principal Components Regression (PCR)](#cell-sovling-Axb-math-pcr)
    5. [SVD Versus Eigendecomposition for PCA/PCR](#cell-sovling-Axb-math-pca-pcr)

2. [Condition](#cell-sovling-condition)
    0. [Matrix Condition Number](#cell-sovling-condition-matrix)
    1. [Scaling / Artifical Ill-Conditioning](#cell-sovling-condition-artificial)
    2. [Pragmatic "Condition" Numbers](#cell-sovling-condition-stiffness)
    3. [Stability and Error Analysis](#cell-sovling-stability)
 
3. [Solving for](#cell-sovling-Axb) [$x$](#cell-sovling-Axb) [in](#cell-sovling-Axb) [$Ax=b$](#cell-sovling-Axb)
  0. [$A^{-1}$](#cell-sovling-AxbwAinv)
    - [Sherman-Morrison-Woodbury Formula](#cell-sovling-SMWoodbury)
    - [[OMITTED] Condition of](#cell-sovling-condition-derivation) [$x=f_{A}(b)=A^{-1}b$](#cell-sovling-condition-derivation)
        - [[OMITTED] Vector and Matrix Norms](#cell-sovling-condition-norms)
    - [[OMITTED] Generalized Inverses](#cell-sovling-inverses)
  1. [Not](#cell-sovling-notAxbwAinv) [$A^{-1}$](#cell-sovling-notAxbwAinv)
    - [[PREREQUISITE] Backward Substitution](#cell-sovling-backsub)
    - [[PREREQUISITE] Gaussian Elimination](#cell-sovling-elimination)
    - [[PREREQUISITE] Elementary Operations](#cell-sovling-elementary)
    0. [The LU Decomposition](#cell-sovling-lu)

  2. [$\text{Finding } X\hat \beta \approx y \text{ by solving for $\hat \beta$ in } \overset{A\, x}{X^T\!X \hat \beta} = \overset{b}{X^t y}$](#cell-sovling-lulike)
    - [[PREREQUESITE] Gradients](#cell-sovling-gradients)
    0. [The $QR$ Decomposition](#cell-sovling-qr)
    1. [The Cholesky Decomposition](#cell-sovling-chol): Week 2 Programming Assignment Problem 2
        - [Application to Covariance Matrices](#cell-sovling-chol-mvn)
    2. [The return of the SVD](#cell-sovling-svd)


In [4]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html
import statsmodels.api as sm
# https://www.statsmodels.org/dev/datasets/index.html
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
# https://en.wikipedia.org/wiki/Variance_inflation_factor#Calculation_and_analysis

<a name="first_cell"></a>

# 3. $Ax=b$ and Singular Values ([Return to TOC](#cell-solving))

---

Linear algebra is concerned with [linear transformations](https://www.mathsisfun.com/algebra/matrix-transform.html) of the form $Ax = b$, and can be viewed as a set of terminologies and notational rules capturing useful concepts and computational operations related to this transformation. One of the centrally useful concepts in linear algebra analysis is ***singular values***. 

Many of the results in statistics are based on specifications representable within the linear algebra framework, so to build a mature level of understanding of statistical methodology it is imperitive to develop comfort and understanding of the underlying linear algebra concepts and notations. 

<a name="cell-transforming-Axb"></a>

# 3.0 Linearly Transforming $x$ with $Ax=b$ ([Return to TOC](#cell-solving))

---

The first interpretation of $A_{n\times m}x_{m\times 1} = b_{n\times 1}$ is that $b$ is a point in $n$-dimensional space which can be expressed in terms of the coordinates $x_j$ with respect to the axes of $m$-dimensional space defined by the column vectors $A_{\cdot j}$ of $A$. That is 

$$b = \underset{\text{$e_i$ are the so-called standard basis vectors}}{b_1 \left[ \begin{array}{c}1\\0\\\vdots\\0 \end{array}\right] + b_2 \left[ \begin{array}{c}0\\1\\\vdots\\0 \end{array}\right] + \cdots + b_n \left[ \begin{array}{c}0\\0\\\vdots\\1 \end{array}\right]} = \sum_i b_i e_{i} = \sum_j x_j A_{\cdot j} = Ax$$

The $A$ matrix specifies the way $x$ is transformed into $b$; or, given $b$, the task may be to find $x$ the alternative coordinate representation of $b$. The inverse problem of solving for $x$ turns out to be very useful.

In [3]:
A = np.ones((3,3)); x = np.ones((3,1))
A@x

In [3]:
A.dot(x)

In [3]:
x.dot(A)

In [9]:
y = np.array([[2],[2],[2]])
# x and y are not linearly independent, since the following is zero.
y - 2*x 

In [9]:
# x and y are not orthogonal since their dot product isn't zero
x.T.dot(y)

In [9]:
xy = np.concatenate([x,y], axis=1)
xy

In [9]:
# the rank of xy is 1 since x and y are linearly dependent
np.linalg.matrix_rank(xy)

In [9]:
e1,e2 = np.array([[1],[0],[0]]), np.array([[0],[1],[0]])
# e1 is a normal vector and is orthogonal to e2 since the following are 0 and 1
e1.T.dot(e1), e1.T.dot(e2)

In [12]:
# the rank of [e1,e2] is 2 since are orthogonal, and hence  linearly independent
np.linalg.matrix_rank(np.c_[e1,e2]) # different way to concatenate

<a name="cell-Axb-etc"></a>

## [PREREQUESITE] Linear Independence, Orthogonality, Rank, and Bases ([Return to TOC](#cell-solving))

---

To uniquely define $b$ above in terms of axes defined by the columns of $A$, it must be the case that that $A_{\cdot j}\not =c A_{\cdot k}$ for all $j \neq k$ so that no two axes redunantly point in the same direction.  This will be the case if the columns $A_{\cdot j}$ are ***linearly indepdendent*** so that

  $$ \underbrace{\sum_{j = 1}^n c_j A_{\cdot j} = 0  \;\; \Longrightarrow  \;\; c_j = 0 \text{ for all } j}_{Ac \;=\; 0 \;\;\Longrightarrow \;\;c \;=\; 0}$$

A stronger condition than ***linear independence*** is ***orthogonality*** where 

$$ (A_{\cdot j})^T A_{\cdot k} =  \sum_i A_{ij} A_{ik} = 0 \text{ for all } j \neq k \quad \text{ and } \quad (A_{\cdot j
})^T A_{\cdot k} \neq 0 \text{ for all } j$$

since for nonzero columns $A_{\cdot j}$ 

$$  \underbrace{(A_{\cdot j})^T A_{\cdot k} = 0}_{\text{Orthogonality}} \quad \Longrightarrow \quad \underbrace{c_jA_{\cdot j} + c_kA_{\cdot k} = 0 \Longrightarrow c_j=c_k=0}_{\text{Linear Independence}}$$

but 

$$\require{\cancel} \underbrace{c_jA_{\cdot j} + c_kA_{\cdot k} = 0 \Longrightarrow c_j=c_k=0}_{\text{Linear Independence}} \quad \cancel{\Longrightarrow} \quad  \underbrace{(A_{\cdot j})^T A_{\cdot k} = 0}_{\text{Orthogonality}}$$

- It is possible to transform two ***linearly independent*** columns $A_{\cdot j}$ and $A_{\cdot k}$ so that they are ***orthogonal***, and the most common way to do this is known as the ***Gram-Schmidt procedure***. Implementation of the ***Gram-Schmidt procedure*** is addressed in the [Week 3 Programming Assignment Problem 1]().

The ***rank*** of the matrix $A_{n\times m}$ is the number of ***linearly independent*** columns (and equivalently, rows) of $A$. The matrix $A$ is said to be ***full rank*** if $\text{rank}(A_{n \times m}) = \min(n,m)$.  When $A$ is ***square*** so $A_{n\times m} = A_{n\times n}$, if $A$ is ***full rank*** then $\text{rank}(A_{n \times n}) = n$ and the $n$ columns of a $A_{n \times n}$ are ***linearly independent*** and form a ***basis*** in $n$-dimensional space. 

- A ***basis*** formed by the $n$ ***linearly independent*** columns of a ***square*** matrix $A$ is a set of axes defining a coordinate system from which to index the $n$-dimensional space.  

  >Any vector of an $n$-dimensional space $x$ may be given in terms of the coordinates of any ***basis*** formed by a ***full rank square matrix*** as 
>
>$$b = \sum_j c_j A_{\cdot j}$$
>
>which illustrates that a ***basis*** does not define the space; rather, the ***basis*** just defines the way points $x$ in the space are referenced.
Changing the ***basis*** does not change the space itself. 

The ***standard basis*** is $A_{n \times n}=I$. The columns of $I$ are the ***standard basis vectors*** $e_j$.  The $e_j$ are ***linearly independent*** and ***orthogonal***; and, because the length of these vectors in the n-dimensional space is 1, they are called ***normal vectors***. 

>The (***Euclidean distance***) length of a vector is given by its ***inner (dot) product*** with itself, so a column vector $A_{\cdot j}$ is a ***normal vector*** if 
>
> $$A_{\cdot j} \cdot{} A_{\cdot j} = \sum_{j = 1}^n = A_{i j}^2 (A_{\cdot j})^T A_{\cdot j} = 1 \quad \text{ e.g., } \quad e_j \cdot e_j = e_j^T e_j =1$$ 

Vectors which are both ***normal*** and ***orthogonal*** are called ***orthonormal***. The ***standard basis*** is thus an ***orthonormal basis***. Two standard convensions that are common in this context are

1. since two vectors are ***linearly independent*** regardless of their length, it is usual to give the vectors of a ***basis*** in their ***normal form***, and
2. an ***orthonormal basis*** is often just called an ***orthogonal basis*** as the ***orthogonality*** is a much more crucial property of such a basis.

<a name="cell-sovling-Axb-math-eig2"></a>

## [PREREQUESITE] Eigenvalues and Eigenvectors ([Return to TOC](#cell-solving))

---

***Eigenvalue*** and ***eigenvector*** analysis of the linear transformation $A_{n\times n}$ examines the rate of the expansion (and/or contraction) along the invariant directions of the transformation, respectively, as

$$A_{n\times n} V_{\cdot j} = \lambda_j V_{\cdot j} \quad \text{often usefully encountered as} \quad (A_{n\times n} - \lambda_j I) V_{\cdot j} = 0$$

Thus, for $x = \sum_{j} c_j V_{\cdot j}$ expressed in an ***eigenvector basis*** (regardless of the ***rank*** of $A_{n\times n}$) 

   $$Ax= A\left(\sum_{j} c_j V_{\cdot j}\right) = \sum_{j} c_j A  V_{\cdot j} = \sum_{j} c_j \lambda_j V_{\cdot j}$$

<a name="cell-sovling-Axb-math-eig1"></a>

## [PREREQUESITE] Eigendecomposition ([Return to TOC](#cell-solving))

---

Any matrix $\Sigma$ such that
\begin{align*}
\Sigma =  {}& \Sigma^T & \textbf{symmetric}\\
x^T\Sigma x {}& > 0 & \textbf{positive definite}\\ 
\end{align*}

is ***full rank*** (so $\text{rank}(\Sigma_{n\times n}) = n$) and may be a ***covariance matrix***. For such matrices, there exists an ***eigendecomposition*** (or synonymously, ***spectral decomposition*** or ***diagonal factorization***)

\begin{align*}
\Sigma_{n\times n} = {} & V_{n\times n} \Lambda_{n\times n} (V^T)_{n\times n}\\
\end{align*}

such that  
- ***orthonormal eigenvectors*** of $\Sigma$ form the columns of the ***orthonormal matrix*** $V_{n\times n}$  


  $$\begin{align*}
  V_{\cdot j}^TV_{\cdot j} & {} = \,\;1\;\,  = V_{j\cdot}^TV_{j\cdot} & {} \textbf{normal vectors}\\
  V_{\cdot j}^TV_{\cdot k} & {} = \,\;0\;\,  = V_{j\cdot}^TV_{k\cdot}, j\not=k & {} \textbf{orthogonality}\\
  V^TV & {} = I_{n\times n}  = VV^T & {} \textbf{orthonormality}
  \end{align*}$$

- and corresponding positive ***eigenvalues*** 

  $$\Lambda_{11}=\lambda_1 \geq \Lambda_{22}=\lambda_2 \geq \cdots \geq \Lambda_{nn}=\lambda_n > 0$$

  comprise the entries of the diagonal matrix $\Lambda_{n\times n}$ 

> [***Eigendecomposition***](https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix) exists more generally for (***square***) [***diagnalizable matrices***](https://math.stackexchange.com/questions/1811983/diagonalizable-vs-full-rank-vs-nonsingular-square-matrix) which might be neither ***positive definite*** nor ***symmetric***. For such matrices, ***eigendecomposition*** can result in ***eigenvalues*** which are negative and ***eigenvectors*** which are not ***orthogonal***:
- the ***eigenvalues*** $\lambda_i > 0$ of $\Sigma$ are positive because $\Sigma$ is **positive definite**
- and the ***eigenvectors*** are ***orthogonal*** because $\Sigma$ is ***symmetric***.
>
> ***Eigendecomposition*** also exists for ***diagonalizable matrices*** which are not ***full rank***. In this case, $r>0$ ***eigenvalues*** will be nonzero $\Lambda_{ii} = \lambda_{i}$ for $i\leq n-r$ and $\Lambda_{ii} = 0$ for $r < i \leq n$ in the diagonal matrix $\Lambda$. The ***eigendecomposition*** then has the ***compact*** form
>
> $$A_{n\times n} = V_{n \times n} \Lambda_{n \times n} V^T_{n \times n} = V_{n \times r} \Lambda_{r \times r} V^T_{r \times n}$$
> 
> and the columns of $V_{n \times r}$ will be ***linearly independent*** [so long as](https://math.stackexchange.com/questions/157382/are-the-eigenvectors-of-a-real-symmetric-matrix-always-an-orthonormal-basis-with) all ***non-zero eigenavalues are unique*** (i.e., have multiplicity $1$). The remaining columns in $V^T_{n \times n}$ may be chosen arbitrarily, e.g., to also be ***linearly independent*** since they will not contribute to $V \Lambda V^T$ for any diagonal element $\Lambda_{ii} = 0$.  

<a name="cell-sovling-Axb-Eigenanalysis"></a>

## Understanding $Ax$ by its Eigenvalues ([Return to TOC](#cell-solving))

---


***Eigenvalues*** determine many properties of an $A_{n \times n}$ matrix.

1. The ***determinant*** of the matrix $A_{n \times n}$ is the product of the ***eigenvalues*** 

   $$\det(A_{n\times n}) = \prod_{i=1}^n \lambda_i$$ 

   and so characterizes the multiplicative change in the "geometric volume" of the space under the linear transformation $Ax$.

2. The ***spectral radius*** of $A_{n\times n}$ is the largest absolute ***eigenvalue***

   $$\rho(A_{n\times n}) = \underset{i=1,...,n}{\max} |\lambda_i| \leq \begin{array}{c}\underset{i=1,...,n}{\max} \sum_{j=1}^n |A_{ij}| \\ \underset{j=1,...,n}{\max} \sum_{i=1}^n |A_{ij}|\end{array}$$
   
   and is the maximul "radius" of the transformation of the space under $A_{n\times n}$ and determines many statistical and computational characteristics of $A_{n\times n}$.

3. The ***trace*** (sum of diagonal elements) of $A_{n\times n}$ is the sum of the ***eigenvalues*** 
  
   $$\text{tr}(A_{n\times n}) = \sum_{i=1}^n A_{ii} = \sum_{i=1}^n \lambda_i$$

   so, e.g., the "total variance" (sum of the diagonal elements) of a ***covariance matrix*** is the sum of the ***eigenvalues*** of the covariance matrix.

<!--
   > which can be shown using the [Jordan canonical form](https://math.stackexchange.com/questions/546155/proof-that-the-trace-of-a-matrix-is-the-sum-of-its-eigenvalues) $A=P J P^{-1}$ (whose diagonal elements $J_{ii}$ are the ***eigenvalues*** of $A$) and the cyclical $\text{trace}(AB)=\text{trace}(BA)$ property of the ***trace*** operator 
   >
   > $$\begin{align*}\text{tr}(A) = {} & \text{tr}(PJP^{-1}) = \text{tr}(JP^{-1}P) = \text{tr}(J) = \sum_{i=1}^n J_{ii} = \sum_{i=1}^n \lambda_i \end{align*}$$
   -->

<a name="cell-sovling-Axb-math-svd"></a>

# 3.1 Singular Value Decomposition (SVD) ([Return to TOC](#cell-solving))

---

For any matrix $A_{n\times m}$ there exists a ***(compact) singular value decomposition (SVD)***

\begin{align*}
A_{n\times m} = {} & U_{n\times r} D_{r\times r} (V^T)_{r\times m}\\
\end{align*}

such that

- $U_{n\times r}$ and $V_{m\times r}$ are ***semi-orthonormal matrices***

  $$\begin{align*}
  U_{\cdot j}^TU_{\cdot j} & {} = \,\;1\;\,  = V_{\cdot j}^TV_{\cdot j} & {} \textbf{normal vectors}\\
  U_{\cdot j}^TU_{\cdot k} & {} = \,\;0\;\,  = V_{\cdot j}^TV_{\cdot k}, j\not=k & {} \textbf{orthogonality}\\
  U^TU & {} = I_{r\times r}  = V^TV& {} \textbf{semi-orthonormality} &\;\quad \text{ so } \quad UU^T \neq I_{r \times r} \left\{\begin{array}{ll} = VV^T & \text{if }r=m\\ \neq VV^T & \text{if }r\neq m\\  \end{array}\right.
  \end{align*}$$

- $D_{r\times r}$ is a ***nonnegative diagonal matrix*** with $D_{ij}=0$ for $i \not = j$ and with so-called ***singular values*** $\lambda_i$ ordered on the diagonal 

  $$D_{11}=\lambda_1 \geq D_{22}=\lambda_2 \geq \cdots \geq D_{rr}=\lambda_r > 0$$

  where 
  
  $$r = \text{rank}(A_{n\times m}) \leq \min(n,m) \quad\quad \text{and} \quad\quad \lambda_j = 0 \text{ for } j>r$$

  which is what gives rise to the ***compact*** version of the ***SVD***.

For ***square*** matrices $A_{n\times m} = A_{n\times n}$ (i.e., $m=n$) we have that the product of the ***singular values*** of $A_{n\times n}$ (not to be confused with the similar formula for ***eigenvalues***) is the ***absolute value of the determinant***

$$|\det(A_{n\times n})| = \prod_{i=1}^n \lambda_i$$

so if $r<n$, then from the ***compact SVD***, $\lambda_j = 0$ for $j = (r+1), ..., n$, then $|\det(A_{n\times n})| = 0$.

> For ***non-square full-rank*** $\tilde A = U\tilde DV^T$ the ***square*** matrix
>
> $$A = \tilde A^T \tilde A = VDU^TUDV^T = V\tilde D \tilde DV^T = V DV^T$$
>
> is ***symmetric*** and ***positive definite*** (and thus satisfies the requirements of ***covariance matrices***).  The $V DV^T$ factorization is the ***SVD*** where $U=V$, but it can also be seen to be the ***eigendecomposition***. Thus, these two factorization are equivalent in the case of ***covariance matrices*** (and the ***singular values*** are equal to the ***eigenvalues***) but this is [not true](https://math.stackexchange.com/questions/127500/what-is-the-difference-between-singular-value-and-eigenvalue) in general. E.g., the ***eigenvalues*** of a ***full-rank square*** matrix may be negative, but ***singular values*** are always non-negative.


In [50]:
Atilde = stats.norm.rvs(size=(10,3))
A = Atilde.T@Atilde
U,d,Vt = np.linalg.svd(A)
print("SVD")
print("U=\n",U,"\nD=\n",np.diag(d),"\nVt=\n",Vt)

eval,evec=np.linalg.eig(A)
# np.linalg.eig does not sort eigenvalues largest to smallest
# the signs on the columns of the eigenvectors can be negated
print("Eigendecomposition")
print("Vt=\n",evec.T,"\nD=\n",np.diag(eval))
# otherwise the factorizations are mathematically equivalent

In [50]:
# the factorizations are no longer mathematically equivalent
A = stats.norm.rvs(size=(3,3))
U,d,Vt = np.linalg.svd(A)
print("SVD")
print("U=\n",U,"\nD=\n",np.diag(d),"\nVt=\n",Vt)
eval,evec=np.linalg.eig(A)
print("Eigendecomposition")
print("Vt=\n",evec.T,"\nD=\n",np.diag(eval))
# e.g., SVD produces real-valued factorizations
# while eigendecompositions are often imaginary

<a name="cell-sovling-Axb-UDVt"></a>

## 3.1.0 Understanding $Ax$ with $Ax=UDV^Tx$ ([Return to TOC](#cell-solving))

---

By examining

$$Ax = (UDV^T)x = \sum_{k=1}^n \lambda_k \left[V^Tx\right]_k U_{\cdot k}$$

it is seen that $Ax$ is actually the weighted sum of the ***orthonormal*** column vectors $U_{\cdot k}$ where the relative contribuition of each depends on the corresponding ***singular value*** $\lambda_k$ and the $k^{th}$ element of $V^Tx$. 


When the ***singular values*** $\lambda_i$ have comparable magnitudes the contributions are on similar scales; but, if the ***singular values*** $\lambda_i$ have different magnitudes the contributions can be on highly different scales. From the perspective of ***numerical accuracy***, adding ***floating-point*** numbers on different scales will result in ***roundoff error***. 

In [69]:
# this results in different magnitude singular values
np.random.seed(2)
n,q=3,10; Atilde = stats.norm.rvs(size=(q,n))*0.01+np.arange(q).reshape(q,1)
A = Atilde.T@Atilde
U,d,Vt = np.linalg.svd(A)
print("Singular Values\n", d)

# this just demonstrates the formula above
x = np.ones((n,1))
print("Ax=\n", A@x)
print("which as a singular values weighted sum is")
weighted_sum = 0*x
for i in range(n):
  weighted_sum += d[i]*(Vt@x)[i]*U[:,[i]]
print(weighted_sum)

<a name="cell-sovling-Axb-math-multicollinearity"></a>

## 3.1.1 Multicollinearity and Variance Inflation Factors (VIFs) ([Return to TOC](#cell-solving))

---

The concept of ***multicollinearity***, the degree to which the covariates correlate, figures prominently in linear model regression contexts, and can be understood in terms of ***singular values***.

For $X_{n\times p} = U_{n\times r} D_{r \times r} (V^T)_{r\times p}$

- if $r<p$ then some columns of the design matrix $X$ are perfectly ***collinear*** and $X$ would not be statistically analyzed without alteration; but,
- even if $r=p$ so $X$ is ***full rank***, if some ***singular values*** $D_{jj}$ are relatively small then the corresponding ***loadings*** $U_{\cdot j}$ and ***basis vectors*** $V_{\cdot j}$ will not contribute significantly to $X$, causing the dominant construction of $X$ to be characteristically  ***non-full rank*** with

$$X_{n\times p} = U_{n\times p} D_{p \times p} (V^T)_{p\times p} \approx U_{n\times r} D_{r \times r} (V^T)_{r\times p} \;\text{ for } r<p$$

Thus the relative magnitudes of the ***singular values*** of $X$ can characterize the degree the ***multicollinearity*** present in $X$.

> The effect of ***multicollinearity*** is best quantified in terms of ***Variance Inflation Factors (VIFs)*** $\frac {1}{1-R_{j}^{2}}$ where
$R_{j}^{2}$ is the ***coefficient of determination*** for the regression of covariate $X_{j}$ on all other covariates except $X_{j}$
> 
> $${\displaystyle X_{j}= \alpha _{0}+\alpha _{1}X_{1} + \alpha _{2}X_{2}\underset{\text{no } X_j}{+\cdots +}\alpha _{k}X_{k}}+e$$
> 
> and is so named since it *inflates* the variation of coefficient estimates according to
> 
> $$ {\displaystyle {\widehat {\operatorname {var} }}({\hat {\beta }}_{j})=s^{2}[(X^{T}X)^{-1}]_{jj} =  {\frac {s^{2}}{(n-1){\widehat {\operatorname {var} }}(X_{j})}}\cdot \underset{VIF}{\frac {1}{1-R_{j}^{2}}}}$$
> 
> where $s^2$ is the ***residual variance*** of the original regression, and the more predictive covariates are of each other the larger $R^2_j$ is and the larger the ***VIF*** is. Thus, the greater the ***multicollinearity*** the greater the uncertainty in coefficient estimation (since attribution between correlated covariates is confounded).

In [281]:
# very low multicollinearity so there's minimal variance inflation
n = 100; X = stats.multivariate_normal(cov=np.array(((1,.09),(.09,1)))).rvs(n)
X = sm.add_constant(X); Y = X.dot(np.ones(3)) + stats.norm().rvs(n)
model = sm.OLS(Y,X); results = model.fit(); results.summary2().tables[1]

In [281]:
# https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html
VIF(X,0),VIF(X,1),VIF(X,2)

In [281]:
U,D,Vt = np.linalg.svd(X, full_matrices=False)
plt.bar(x=range(3), height=D); plt.title("Singular Values"); 

In [281]:
# This shows that the approximate reconstruction of X is very poor 
fix,ax = plt.subplots(1,2, figsize=(10,4))
ax[0].hist(np.abs(U.dot(np.diag(D)).dot(Vt) - X).flatten())
ax[0].set_title("Roundoff error in full reconstruction of X")
ax[1].hist(np.abs(U[:,:2].dot(np.diag(D[:2])).dot(Vt[:2,:])-X).flatten())
ax[1].set_title("Approximation error in compact reconstruction of X");

In [281]:
plt.plot((U[:,:2].dot(np.diag(D[:2])).dot(Vt[:2,:])).ravel(), X.ravel(),'.',
         label='Approximation error in compact reconstruction of X')
plt.plot((U.dot(np.diag(D)).dot(Vt)).ravel(), X.ravel(),'.',
         label='Roundoff error in full reconstruction of X')
plt.title("X is poorly constructed with a lower rank approximation")
plt.xlabel("Exact X"); plt.xlabel("Reconstructed X"); plt.legend();

In [281]:
# The standard errors are inflated for highly multicolliner X
n = 100; X = stats.multivariate_normal(cov=np.array(((1,.99),(.99,1)))).rvs(n)
X = sm.add_constant(X); Y = X.dot(np.ones(3)) + stats.norm().rvs(n)
model = sm.OLS(Y,X); results = model.fit(); results.summary2().tables[1]

In [281]:
VIF(X,0),VIF(X,1),VIF(X,2)

In [281]:
U,D,Vt = np.linalg.svd(X, full_matrices=False)
plt.bar(x=range(3), height=D); plt.title("Singular Values"); 

In [281]:
# This shows that the approximate reconstruction of X is not as bad
# when one of the singular values is small and dropped from the SVD
fix,ax = plt.subplots(1,2, figsize=(10,4))
ax[0].hist(np.abs(U.dot(np.diag(D)).dot(Vt) - X).flatten())
ax[0].set_title("Roundoff error in full reconstruction of X")
ax[1].hist(np.abs(U[:,:2].dot(np.diag(D[:2])).dot(Vt[:2,:])-X).flatten())
ax[1].set_title("Approximation error in compact reconstruction of X");

In [281]:
plt.plot((U[:,:2].dot(np.diag(D[:2])).dot(Vt[:2,:])).ravel(), X.ravel(),'.',
         label='Approximation error in compact reconstruction of X')
plt.plot((U.dot(np.diag(D)).dot(Vt)).ravel(), X.ravel(),'.',
         label='Roundoff error in full reconstruction of X')
plt.title("X is fairly well constructed with a lower rank approximation")
plt.xlabel("Exact X"); plt.xlabel("Reconstructed X"); plt.legend();

<a name="cell-sovling-Axb-math-pca"></a>

## 3.1.2 Principal Components Analysis (PCA) ([Return to TOC](#cell-solving))

---

[***Principal Components Analysis***](https://en.wikipedia.org/wiki/Principal_component_analysis) (***PCA***) is an ***unsupervised learning*** methodology which uncovers linear structure underlying a data matrix $X_{n \times m}$. When doing ***PCA***, the "principal components are often computed by eigendecomposition of the data covariance matrix or singular value decomposition of the data matrix". So 
> for the usual ***PCA*** standardization preprocessing, where the data columns of $X_{n \times m}$ are centered and scaled by their column means $\overline{X_{\cdot j}}$ and standard deviations $\hat \sigma_{X_{\cdot j}}$ as $\tilde X_{\cdot j}  = (X_{\cdot j} - \overline{X_{\cdot j}}) / \hat \sigma_{X_{\cdot j}}$ 

a ***PCA*** is computed as either

  $$\underbrace{\tilde X \div \sqrt{n} = U_{n \times m}D_{m \times m}(V^T)_{m \times m}}_{\text{singular value decomposition of the data matrix}}\quad\text{ or }\quad
\underbrace{\hat \Sigma = (\tilde X^T\tilde X \div n) = V_{m \times m}D^2_{m \times m}V^T_{m \times m}}_{\text{eigendecomposition of the data covariance matrix}}$$

which explicitly demonstrates [the connection](https://math.stackexchange.com/questions/3869/what-is-the-intuitive-relationship-between-svd-and-pca) between ***SVD*** and the ***eigendecomposition***. Namely, for the ***SVD*** $X=UDV^T$, the ***eigendecomposition*** of the so-called ***gramian matrix***

$$X^TX = V D^2 V^T$$

has ***eigenvalues*** which are the square of the ***singular values*** of $X$ and has the same ***(semi-)orthonormal*** matrix $V^T$ as ***SVD*** (where $V^T$ will be ***orthonormal*** if $X$ is ***full rank*** so the ***gramian*** is ***positive definite***).

In [329]:
# This is an example of a PCA analysis
mtcars = sm.datasets.get_rdataset("mtcars")
Xtilde = (mtcars.data - np.mean(mtcars.data))/np.std(mtcars.data)
# The actually PCA computation is the next line
U,D,Vt = np.linalg.svd(Xtilde/np.sqrt(Xtilde.shape[0]))

# Below is the code for standard PCA analyses for so called "Principle Components"
# The i^th column of U from the SVD is the i^th Principle Component
# A compact SVD approximate reconstruction of Xtilde would use 1st through i^th Principle Components
plt.figure(figsize=(18,6))
plt.subplot(131); plt.plot(D**2,'.'); plt.plot(D**2); plt.xlabel("Principle Component");
plt.ylabel("Singular Values of ~X"); plt.title("Variation Explained by Each Principle Component")
plt.subplot(132); 
p=2; plt.plot(U[:,:p].dot(np.diag(D[:p])).dot(Vt[:p,:]).flatten(), 
         Xtilde.values.flatten()/Xtilde.shape[0]**.5, '.', 
         label="Rebuild ~X with 2 PCs")
p=4; plt.plot(U[:,:p].dot(np.diag(D[:p])).dot(Vt[:p,:]).flatten(), 
         Xtilde.values.flatten()/Xtilde.shape[0]**.5, '.', 
         label="Rebuild ~X with 4 PCs")
p=11; plt.plot(U[:,:p].dot(np.diag(D[:p])).dot(Vt[:p,:]).flatten(), 
         Xtilde.values.flatten()/Xtilde.shape[0]**.5, '.', 
         label="Rebuild ~X with 11 PCs")
plt.legend(); plt.xlabel("Rebuilt value"); plt.ylabel("Original value"); 
plt.title("Reconstructing ~X from its Principle Components")
plt.subplot(133); plt.plot(U[:,0],U[:,1],'.'); 
plt.xlabel("Principle Component 1"); plt.ylabel("Principle Component 2");
plt.title("Interpreting Principle Components of ~X");
for i in range(0, U.shape[0]):
  if all(np.sum(((U[:i,:2]-U[i,:2])**2).dot([[1],[15]]), axis=1) > .02):
    plt.text(U[i,0],U[i,1],mtcars.data.index[i], horizontalalignment="right")
for j in range(Vt.shape[0]):
  plt.plot([0,Vt[0,j]],[0,Vt[1,j]],'k')
  plt.text(Vt[0,j], Vt[1,j], mtcars.data.columns[j], color='r')
# The third plot interprets the "Principle Directions" associated with each "Principle Component"
# Consdier the Xtilde space of mpg, cyl, disp, etc.
# In this space the first "Principle Component" (the first column of U from the SVD) 
# points most directly in the direction of the cyl and disp axes, and in the negative direction of the mpg axis
# The elements of the first column of U are the coordinates of the data points along this direction
# The blue points are data points, and you can see their locations relative to the Priniple Directions

In [329]:
# Xtilde is standardized, so var for each col is 1
np.var(Xtilde).sum(), (D**2).sum()
# D are the singular values of Xtilde 
# D**2 are the eigenvalues of covariance matrix from its eigendecomposition, 
#      which is equivalent to the singular values of the covariance matrix
# The sum of the eigenvalues of cov matrix is trace of the cov matrix 
# which is the sum of the variances for each column in the standardized Xtilde

***PCA*** is the ***SVD*** of $X$ which provides the principle directions in $X$ and the projections of $X$ onto this coordinate coordinate system are the ***principle components*** of $X$.

![](https://miro.medium.com/max/596/1*QinDfRawRskupf4mU5bYSA.png)

<a name="cell-sovling-X-UDVt"></a>

## 3.1.3 Understanding PCA as $X=UDV^T$ ([Return to TOC](#cell-solving))

---

For ***full rank*** $X_{n \times m}$, the ***SVD*** $X=UDV^T$ produces an ***orthonormal*** $V^T$ so that the data point

$$X_{i \cdot} = U_{i \cdot} DV^T \quad \text{ or } \quad x_i = \underbrace{VD}_{A}u_i$$ 

can be be represented by $U_{i \cdot}$ with respect to the ***orthogonal basis*** defined by the columns of $A=VD$. The $X_{i \cdot}$ data point is represented in the ***standard (orthonormal) basis***, but the columns $X_{\cdot j}$ are not necessarily ***linearly independent***; however, $U_{i \cdot}$ is represented in a different ***orthogonal basis*** where the columns $U_{\cdot j}$ are ***linearly independent*** since they are ***semi-orthonormal*** ($U^TU = I_{m \times m}$).

> Since a point in $m$-dimensional space can be equivalently defined by different ***bases***, the rows of $X$ and $U$ represent the same collection of points under different ***bases***; but, the fact that $U$ has ***linearly independent*** columns means that it has no ***multicollinearity***.

- Each column $U_{\cdot j} = \left[X  V D^{-1}\right]_{\cdot j} = \sum_{k=1}^m X_{\cdot k} V_{k j}/\lambda_j$ is a linear combination of the columns of $X$, so $U_{\cdot j}$ are new variables constructed as weighted averages of the columns of $X$. 

- Each $X_{\cdot j} = \left[U D V^T \right]_{\cdot j} = \sum_{k=1}^m \lambda_k U_{\cdot k} [V^T]_{k j} \approx \sum_{k=1}^r \lambda_k U_{\cdot k} [V^T]_{k j}$ is a linear combination of the columns of $U$, where the summation to $r<m$ shows $X_{n \times m}$ approximated as a lower rank reconstruction using the ***compact SVD*** form. 



In [329]:
i=1; X = stats.norm.rvs(size=(5,3))
U,d,Vt = np.linalg.svd(X, full_matrices=False)
print(X[i,:], "\n", (U[i,:]*d)@Vt)
# U[i,:]*d is the "broadcasted" version of U[i,:]@np.diag(D)
# https://numpy.org/doc/stable/user/basics.broadcasting.html

In [329]:
j=0
print(X @ (Vt.T)[:,[j]] / d[j])
print(U[:,[j]])

In [329]:
j=1
print(X[:,[j]])
print((U*d)@Vt[:,[j]])

<a name="cell-sovling-Axb-math-pcr"></a>

## 3.1.4 Principal Components Regression (PCR) ([Return to TOC](#cell-solving))

---

In addition to use as an ***unsupervised learning*** methodology, ***PCA*** can be applied in the context of linear regression in order to replace a ***linearly dependent*** $X$ with ***linearly independent (semi-orthogonal)*** $U$ which is either immediately available from ***SVD*** or is readily computed from $V$ and $D$ of the ***eigendecomposition*** as 

$$X = UDV^T \Longrightarrow \underbrace{XVD^{-1} = UDV^TVD^{-1} = U}_{\text{exressed for a single data point }D^{-1}V^Tx_i = u_i}$$

While the general motivation to replace $X$ with $U$ is to avoid the attribution uncertainty associated with ***multicollinearity***, and where possible to improve interpretability (through "factors" of linear combinations of the original columns of $X$), $U$ is also preferable to $X$ since all it's ***singuar values*** are $1$ (as seen from the ***SVD*** of $U$ which is $U = U II$).

- All of these benefits are realized by viewing $x_i = Au_i$ as a ***linear transformation*** so each data point $x_i$ is represented by $u_i$ in the ***orthogonal basis*** $A=VD$ rather than the ***standard (orthonormal) basis***.

In [42]:
n = 100; X = stats.multivariate_normal(cov=np.array(((1,.999),(.999,1)))).rvs(n)
X = sm.add_constant(X); Y = X.dot(np.ones(3)) + stats.norm().rvs(n)
model = sm.OLS(Y,X); results = model.fit(); results.summary2().tables[0] # scale = residual variance
#float(results.summary2().tables[0].iloc[-1,-1])**.5

In [42]:
results.summary2().tables[1]

In [42]:
VIF(X,0),VIF(X,1),VIF(X,2)

In [42]:
U,D,Vt = np.linalg.svd(X, full_matrices=False)
plt.bar(x=range(3), height=D); plt.title("Singular Values"); 

In [42]:
# but now we use U rather than X -- same model fit accuracy scores
model = sm.OLS(Y,U); results = model.fit(); results.summary2().tables[0] # scale = residual variance
#float(results.summary2().tables[0].iloc[-1,-1])**.5

In [42]:
# but tighter standard errors
results.summary2().tables[1]

In [42]:
# because now there's no multicollinearity, so VIFs are 1
VIF(U,0),VIF(U,1),VIF(U,2)

In [42]:
# and the SVD of U=U*I*I so singular values of U are all 1
U2,D,Vt = np.linalg.svd(U, full_matrices=False)
plt.bar(x=range(3), height=D); plt.title("Singular Values"); 

<a name="cell-sovling-Axb-math-pca-pcr"></a>

## 3.1.5 SVD Versus Eigendecomposition for PCA/PCR ([Return to TOC](#cell-solving))

---

The ***PCA*** analyses above were created on the basis of an ***SVD*** computation; however, as discussed in this [stackoverflow](https://stats.stackexchange.com/questions/314046/why-does-andrew-ng-prefer-to-use-svd-and-not-eig-of-covariance-matrix-to-do-pca) conversation, there are (at least) four distinct algorithms which could be used to compute $V^T$ and $D$ (and thus compute ***PCA***):

- `np.linalg.svd(X)`
- `np.linalg.svd(X^TX)`
- `np.linalg.eig(X^TX)`
- `np.linalg.eigh(X^TX)`

It turns out that `np.linalg.svd(X)` is the best choice for two reasons:

1. All computations generally entail ***roundoff error***, so calculations such as $X^TX$ should be avoided for numerical accuracy when possible.
  - Even the ***singular values***/***eigenvalues*** $D_{ii}^2$ (of $X^TX$) tend to be less precice compared to the numerical accuracy of ***singular values*** $D_{ii}$ (of $X$) since squaring introducess the possibility for ***roundoff error***.
3. The ratio of the largest to smallest ***singular values*** of $X$ can be small compared to $X^TX$ since

   $$\frac{\max_i D_{ii}}{\min_j D_{jj}} << \frac{\max_i D_{ii}^2}{\min_j D_{jj}^2}$$

   since if the the ***spectral radius***
   
   $${\rho(X) = \max_i D_{ii}} > 1 > \min_j D_{jj}$$

   then the ***spectral radius*** of the ***gramian***

   $${\rho(X^TX) = \max_i D_{ii}}^2 >> 1 >> \min_j D_{jj}^2$$

  which is relevant for the topic of ***condition***, introduced next.

In [42]:
np.set_printoptions(precision=7)

n,m = 100,7
X = stats.norm().rvs(size=(n,m))
X = (X - X.mean(axis=0))/X.std(axis=0)
print("The total variance", X.var(axis=0).sum(), end=" ")
print("should match the sum of the eigenvalues") 
print("which is also equal to the sum of squares of singular values.\n")

# Traditional PCA: eigendecomposition of the covariance matrix
# np.linalg.eigh: "complex Hermitian (conjugate symmetric) or a real symmetric matrix."
eignvals = np.sort(np.linalg.eigh(np.cov(X.T, ddof=0))[0])[::-1]
print(eignvals, eignvals.sum())

# Traditional PCA: eigendecomposition of the covariance matrix
# np.linalg.eig: general eigendcomposition function
eignvals = np.sort(np.linalg.eig(np.cov(X.T, ddof=0))[0])[::-1]
print(eignvals, eignvals.sum())

# Traditional PCA: eigendecomposition of the covariance matrix
# SVD/eigendecomposition equivalence for symmetric positive definite matrices
eignvals = np.linalg.svd(np.cov(X.T, ddof=0))[1]
print(eignvals, eignvals.sum())

# Traditional PCA: eigendecomposition of the covariance matrix
# np.linalg.eigh with matrix-based covariance computation
eignvals = np.sort(np.linalg.eigh(X.T.dot(X)/n)[0])[::-1]
print(eignvals, eignvals.sum())

# Traditional PCA: eigendecomposition of the covariance matrix
# np.linalg.eig with matrix-based covariance computation
eignvals = np.sort(np.linalg.eig(X.T.dot(X)/n)[0])[::-1]
print(eignvals, eignvals.sum())

# Traditional PCA: eigendecomposition of the covariance matrix
# np.linalg.eig with matrix-based covariance computation
eignvals = np.linalg.svd(X.T.dot(X)/n)[1]
print(eignvals, eignvals.sum())

# PCA with SVD 
singvals = np.linalg.svd(X/n**0.5)[1]
print(singvals**2, (singvals**2).sum())

<a name="cell-sovling-condition"></a>

# 3.2 Condition ([Return to TOC](#cell-solving))

---

The ***condition*** of a mathematical function $f$ addresses whether 

$$\underset{\large \text{$f$ is well-conditioned}}{x+\epsilon_x \approx x \Longrightarrow f(x+\epsilon_x) \approx f(x)} \quad \text{ or } \quad \underset{\large \text{$f$ is ill-conditioned}}{x+\epsilon_x \approx x \cancel{\Longrightarrow} f(x+\epsilon_x) \approx f(x)}$$

***Condition*** is a mathematical property of a problem, but small changes $\epsilon_x$ in the input of a problem $x$ can be caused by ***roundoff error***, so the ***condition of a problem*** is a key issue for ***numerical accuracy***.

In [91]:
# this example setup was used before
np.random.seed(2)
n,q=3,10; Atilde = stats.norm.rvs(size=(q,n))*0.01+np.arange(q).reshape(q,1)
A = Atilde.T@Atilde
U,d,Vt = np.linalg.svd(A)
print("Singular Values of A", d, "\n")

# Now we'll see what Ax is like versus A(x+epsion) for some small epsilon
x1 = Vt[[2],:].T
epsilon = Vt[[0],:].T/1000
x2 = x1+epsilon
diff = (x1-x2)
print("Difference between x and x+epsion", (diff.T@diff)**0.5)
diff = (A@x1-A@x2) 
print("Difference between A(x) and A(x+epsion)", (diff.T@diff)**0.5)
# very different for such a small difference in the input...

<a name="cell-sovling-condition-matrix"></a>

## 3.2.0 Matrix Condition Number ([Return to TOC](#cell-solving))

---

In the context of $f_A(x)=Ax=b$,  the ***condition*** of $f_A$ depends on the relative magnitudes of the ***singular values*** of the $A$ matrix in

$$Ax = (UDV^T)x = \sum_{k=1}^n \lambda_k \left[V^Tx\right]_k U_{\cdot k}$$

This is because $\left[V^Tx\right]_k$ is the dot product of the $k^{th}$ column of $V$ with $x$ which will be $0$ if these vectors are ***orthogonal***; but, if they are not ***orthogonal*** then this dot product is scaled by $\lambda_k$ which could be very large. If all $\lambda_{k'\neq k}$ are very small, the $\lambda_k \left[V^Tx\right]_k$ multiplier could thus shift from $0$ to a large nonzero on the basis of some small $\epsilon>0$. This is exactly what is demonstrated in the code above.

$$\underset{\text{if the singular values of $A$ have similar small magnitudes}}{\overset{\Large \text{$f_A$ is well-conditioned}}{x+\epsilon_x \approx x \Longrightarrow f_A(x+\epsilon_x) \approx f_A(x)}} \quad \text{ or } \quad \underset{\text{if the singular values of $A$ don't have similar magnitudes}}{\overset{\Large \text{$f_A$ is ill-conditioned}}{x+\epsilon_x \approx x \cancel{\Longrightarrow} f_A(x+\epsilon_x) \approx f_A(x)}}$$


Since the relative spacing of ***singular values*** is bounded by the smallest and largest ***singular values***, an easy useful general ***matrix condition number*** is just $\kappa(A) = \frac{\lambda_{max}}{\lambda_{min}}$ the ratio of the largest and smallest ***singular values***. If this  ***matrix condition number*** is small then the matrix is said to be ***well-conditioned*** while if its large then the matrix is said to be ***ill-conditioned***.

- The spacing of the ***singular values*** of the ***gramian*** $X^TX = V D^2 V ^T$ will be much more spread out than those of $X= U D V ^T$ itself since they are squared. It's often therefore the case that 
  
   $$\kappa(X^TX) = \frac{\lambda_{max}^2}{\lambda_{min}^2} >> \frac{\lambda_{max}}{\lambda_{min}} = \kappa(X)$$

  so the ***condition number*** of the ***gramian*** $X^TX$ is *much* larger than the ***condition*** of $X$ itself.

  > In ***PCA***, because of the generally increased ***condition number*** of the ***covariance matrix*** $\tilde X^T \tilde X$ relative to the scaled data matrix $\tilde X$, applying ***SVD*** to $\tilde X$ is more numerically stable than computing the ***eigendecomposition*** or ***SVD*** of $\tilde X^T \tilde X$.


- Constituant components of matrix decompositions like the ***orthogonal matrix*** $U$ in ***SVD*** $ A = U D V^T$ have smaller ***condition numbers*** than the original matrix $A$ because they factor out the ***singular values*** of $A$. For ***orthogonal matrix*** $U$ in ***SVD*** $ A = U D V^T$  

   $$U=U\Lambda W^T = U_{n\times m}I_{m \times m}I_{m \times m}$$
   
   so all ***singular values*** of the ***SVD*** $U$ are $\lambda_i = \Lambda_{ii} = I_{ii} = 1$ and $\kappa(U) = 1$.

  > In the context of linear regression, the ***condition*** of the  principal components of the ***SVD*** $X = UXV^T$ is $1=\kappa(U) \leq \kappa(X)$; thus, ***principle components regression*** is a mathematical formulation which specifies the use of a ***well-conditioned*** design matrix.

In [ ]:
# Condition Numbers are a good check for linear models
mtcars = sm.datasets.get_rdataset("mtcars")
Y = mtcars.data[['mpg']].values
X = mtcars.data[['cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear','carb']].values
X = sm.add_constant(X)
model = sm.OLS(Y,X); results = model.fit(); results.summary2()

In [ ]:
# Confirmed
D = np.linalg.svd(X)[1]
D[0]/D[-1]

In [ ]:
# Confirmed
np.linalg.cond(X)

In [ ]:
# breaking the condition number
np.linalg.svd(np.ones((2,2)))[1], np.linalg.cond(np.ones((2,2)))

<a name="cell-sovling-condition-artificial"></a>

## 3.2.1 Scaling / Artifical Ill-Conditioning ([Return to TOC](#cell-solving))

---

Matrices can be ***artifically ill-conditioned*** if the scales of the columns (or rows) differ substantially from one another; but, this can be corrected by simply re-scaling the columns (or rows) of the matrix in question.

- The second instruction of the ever-present recommendation to "center and scale your data" in linear regression contexts is thus seen to address the ***artifical ill-conditioning*** problem. 

- Systems of nonlinear equations will be considered later, but one approach to solving them is to use a ***multivariate first-order Taylor approximation*** to transform the problem back into a system of linear equations $$b = f(x) \approx f(x^{(t)}) + \overbrace{\left[\begin{array}{ccc}\frac{\partial f_1(x^{(t)})}{\partial x_1} & \cdots & \frac{\partial f_1(x^{(t)})}{\partial x_p} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_n(x^{(t)})}{\partial x_1} & \cdots & \frac{\partial f_n(x^{(t)})}{\partial x_p} 
\end{array} \right]}^{\text{the Jacobian } J_{f(x)}(x^{(t)}) \,=\, \nabla_{f(x)}(x^{(t)})^T}  (x-x^{(t)})$$

    However, $J\tilde x = \tilde b$ will be ***artifically ill-conditioned*** if the scales of the ***Jacobian*** differ greatly from, e.g., row to row. If one row of parital derivatives (i.e., one particular outcome variable of $f$) has substantially different magnitudes from the other rows, the equations can be rescaled, i.e., to $c_ib_i = c_if_i(x)$, so that there are not drastically different magnitudes from one row of the  ***Jacobian*** to the next.

<a name="cell-sovling-condition-stiffness"></a>

### 3.2.2 Pragmatic "Condition" Numbers ([Return to TOC](#cell-solving))

---

In it's most technically correct sense, the ***condition of a problem*** characterizes how dramatically a function output changes for "nearby inputs"; however, since a primary driver of small variation in input is numerical imprecision from ***roundoff error***, some ***condition numbers*** simply characterize when a computation becomes "numerically unstable" solely as a result of numerical issues. An example of this kind of ***condition number*** is the so-called ***stiffness*** of a sample in the context of a variance calculation

$$\begin{align*}\kappa(x) = {} & \frac{\sum_{i=1}^n x_i^2}{s\sqrt{n-1}} = \frac{\sum_{i=1}^n x_i^2}{\sum_{i=1}^n x_i^2 - n\left(\frac{\sum_{i=1}^n x_i}{n}\right)^2}\\
\text{where } s^2 = {} & \frac{\sum_{i=1}^n (x_i-\bar x)^2}{n-1}\\
\end{align*}$$

The ***stiffness condition number*** $\kappa(x) \approx 1$ when $\bar x \approx 0$ but $\kappa(x) >> 1$ when $\bar x >> s$, which is exactly when a sample variance calculation can fail numerically (and is yet another example of why the **"center and scale your data" mantra** is indeed good practice).

- Sample variance calculations were previously illustrated to fail as a result of ***Catastrophic Cancellation***, and an alternative sample variance calculation algorithm with reduced ***roundoff error*** was explored in the [Week 2 Programming Assignment]().

Pragmatically, both ***stiffness*** and ***condition number*** $\kappa(A)$ address equivalent objectives of identifying if the computations of a function are expected to be "numerically volatile". 

<a name="cell-sovling-stability"></a>

## 3.2.3 Stability and Error Analysis ([Return to TOC](#cell-solving))

---

There are several additional terms that are related to the ***condition*** of a problem.

- ***Robust*** algorithms (programs) either 
  - warn of the presence of ***ill-conditioning*** and may or may not attempt to compute an answer
  - or (for ***well-conditioned problems***) give answers that are as numerically correct as possible under the available ***precision***.

- The computation $\tilde f(\cdot)$ calculating $f(\cdot)$ is (backward) ***stable*** if the answers are always true solutions to <u>existing</u> "nearby problems":

  $$\tilde f(x) \neq f(x) \quad \text{ but } \quad f(\tilde x = x+\epsilon_x) = \tilde f(x)$$

  - Note that ***stability*** does not overcome ***ill-conditioned problems*** since
  
    $$f(\tilde x = x+\epsilon_x) = \tilde f(x) \cancel{\Longrightarrow} f(x+\epsilon_x) \approx f(x)$$

   **Stability** only means that computations are "close" as a matter of the  numerical ***precision*** of the input; but **stability** says nothing about the ***condition*** of a problem. That is, the precison of the computation $\tilde f(\tilde x) \approx f(\tilde x)$ does not mean these values are close to $f(x=\tilde x-\epsilon_x)$. 
   

<!--
    $$b+\epsilon_b \approx b \cancel{\Longrightarrow} f_A(b+\epsilon_b) \approx f_A(b)$$
    > E.g., $x = f_A(b) = A^{-1}b$; but, if $\kappa(A)$ is large, then $\frac{||\epsilon_x||}{||x||}$ may be large even if $\frac{||\epsilon_b||}{||b||}$ is small, since $\frac{||\epsilon_x||}{||x||} \leq \kappa(A) \frac{||\epsilon_b||}{||b||}$ so 
  >
  > $$b+\epsilon_b \approx b \cancel{\Longrightarrow} f_A(b+\epsilon_b) \approx f_A(b) $$
-->

<!--
  > and no "nearby problem" may exist so
  >
  > $$\underset{\text{for all choices of small $\epsilon_b$ and $\epsilon_x$} }{f_A(b+\epsilon_b) \not = \tilde f_A(b) + \epsilon_x}$$

  > $$\require{cancel} \underset{ \text{where $f(\tilde b)\\, \approx f(b)$ if $f$ is ill-conditioned} }{\overset{\text{there are no nearby problems } \tilde b \\, = \\, b+\epsilon_b}{\cancel{f(\tilde b = b+\epsilon_b) = f(b) + \epsilon}}}$$

-->

<!-- $\require{enclose} \enclose{horizontalstrike}{}$ -->

| |
|-|  
|![Nick Higham's Blog](https://nickhigham.files.wordpress.com/2020/03/berr-fig-2.jpg?w=500&zoom=2)|
|Taken from Nick Higham's website: https://nhigham.com/2020/03/25/what-is-backward-error/|

- ***Forward error analysis*** compares the difference between computed and true answers to problems. However, since computation is done in ${\rm I\!F}$ and the correct answers in ${\rm I\!R}$ may be neither known nor representable in ${\rm I\!F}$, this is not generally applicable.
- ***Backward error analysis***, in contrast, is based fully on computations within ${\rm I\!F}$. For 

  - $b, \tilde b=b+\epsilon_b \in {\rm I\!F}$
  - $\tilde f(b)$ the computed version of the true answer $f(b)$
  - $\tilde f(b) = f(\tilde b) = \tilde f(\tilde b)$ 
    
    > i.e., $\tilde f(\tilde b)$ is computed exactly correctly as  $f(\tilde b)$ which has the same value as the other computed value $\tilde f(b)$ (which is not necessarily computed correctly)

  ***backward error analysis*** evaluates how large $\epsilon_b$ the difference between $b$ and $\tilde b$ is.

  > It is possible to check if $\tilde f(\tilde b) = f(\tilde b)$, i.e., if the output of a computation is exactly represented (by checking if ***roundoff*** error was encountered during the calculation); thus, if $b$ is found such that $\tilde f(b) = \tilde f(\tilde b) = f(\tilde b)$, then the difference between (***floating point numbers***) $b$ and $\tilde b$ can of course be evaluated.

***Backward error analysis*** can be seen to be defined in the same terms as (backward) ***stability*** since

- if $\epsilon_b$ is small where $\tilde f(b) = f(\tilde b = b + \epsilon_b)$
  > i.e., $\tilde f(b)$ is the answer to the $\epsilon_b$-close "nearby problem"
- then the algorithm is (backward) ***stable*** (at $b$) since answers here are true solutions to an existing "nearby problem". 

<a name="cell-sovling-Axb"></a>

# 3.3 Solving for $x$ in $Ax=b$ ([Return to TOC](#cell-solving))

---

When $A^{-1}$ exists so that there is a solution $x$ for $Ax=b$, rather than computing 

- $x=A^{-1}b$ with `np.linalg.inv(A) @ b` 

a better solution  

- solving for $x$ in $Ax=b$ with `np.linalg.solve(A, b)`

does not require explicitly computing $A^{-1}$, and it is faster to not do so.

In [86]:
n = 1000
A, b = stats.norm.rvs(size=(n,n)), stats.norm.rvs(size=(n,1))

In [89]:
%timeit np.linalg.inv(A) @ b

In [89]:
# 3 times faster (not n times faster)... we'll return to this later
%timeit np.linalg.solve(A, b)

In [89]:
# and of course if A isn't invertible, then...
np.linalg.inv(np.ones((n,n)))

<a name="cell-sovling-AxbwAinv"></a>

## 3.3.0 $A^{-1}$ ([Return to TOC](#cell-solving))

---

If we write $x=A^{-1}b$ as the solution to the system of linear equations $Ax = b$ we are implying that $A$ is an $n\times n$ square matrix which is ***full rank*** or (synonymously) ***invertible or nonsingular***. $A^{-1}$ doesn't exist for ***non full rank*** or (synonymously) ***non-invertible or singular*** matrices.

> The use of the synonym ***nonsingular*** in place of the more straightforward term ***invertible*** is because if $A^{-1}$ does not exist, then the ***inverse function***
>
> $$f(A) = A^{-1}$$
>
> is not defined at $A$ and so then $A$ is a point of [mathematical singularity](https://en.wikipedia.org/wiki/Singularity_(mathematics)) in the ***domain*** of $f$. 

$A_{n\times n}^{-1} = $ [$\det(A_{n\times n})^{-1}\operatorname {adj}(A_{n\times n})$](https://en.wikipedia.org/wiki/Adjugate_matrix#Definition) and the ***determinant*** is the product of the ***eigenvalues*** of $A$ a well as the product of the (absolute) ***singular values*** of $A$. Thus, for ***singular values*** (and ***eigenvalues***) of $A_{n\times n}, \lambda_j, 1 \leq j \leq n$, if

| $\lambda_j = 0$ for some $j$| $\lambda_j \not = 0$ for all $j$ |
|-|-|
| $\det A = 0$ | $\det A \neq 0$ |
| division by $0$ | no division by $0$ | 
|$A$ is ***singular*** | $A$ is ***nonsingular*** |
| $A$ is ***not invertible*** | $A$ is ***invertible*** |
| $A$ is ***not full rank*** | $A$ is ***full rank*** |
| Some columns (rows) are | All columns (rows) are
| ***linearly dependent*** | ***linearly independent*** |
| $A^{-1}$ does not exist | $A^{-1}$ exists |

Even if $A^{-1}$ exists, there are three problems:

0. Inverse computation is **(usually)** not a simple algorithm, like ***transpose*** $A^T$ 
  - which just reverse the indexing scheme $\quad[A^T]_{ij} = A_{ji}$
  - and have simple higher order properties $\quad (AB)^T = B^TA^T$

    > However, notice that for ***orthonormal*** matrices (which are often simply just referred to as ***orthogonal*** matrices since the columns can be easily ***standardized*** into ***normal vectors***)
    > $$W_{n \times n}^TW_{n \times n}=W_{n \times n}W_{n \times n}^T = I_{n \times n}.$$    
    >
    > and **inversion is transposition**; and, this is partially true for 
    > ***semi-orthogonal*** (or ***semi-orthonormal***) matrices where 
    >
    >   $$S_{n \times m}^TS_{n \times m}=I_{m \times m}$$

1. Inverse computation is unnecessarily wasteful
  - since solving for $x$ in $Ax = b$ means solving $$A \left[\begin{array}{c}x_1\\\vdots\\x_n\end{array}\right] = \left[\begin{array}{c}b_1\\\vdots\\b_n\end{array}\right]$$
  - but computing $x=A^{-1}b$ means either knowing or solving for $A^{-1}$
  $$A \left[\!\!\!\!\!\!\begin{array}{c:c:c:c} & 
  \begin{array}{c}A_{11}^{-1}\\\vdots\\A_{n1}^{-1}\end{array}& 
  \begin{array}{c}A_{12}^{-1}\\\vdots\\A_{n2}^{-1}\end{array}&\cdots&
  \begin{array}{c}A_{1n}^{-1}\\\vdots\\A_{nn}^{-1}\end{array}& 
  \end{array}\!\!\!\!\!\!\right] = 
  \left[\!\!\!\!\!\!\begin{array}{c:c:c:c} & 
  \begin{array}{c}1\\0\\\vdots\\\vdots\\0\end{array}& 
  \begin{array}{c}0\\1\\0\\\vdots\\0\end{array}&\cdots&
  \begin{array}{c}0\\\vdots\\\vdots\\0\\1\end{array}& 
  \end{array}\!\!\!\!\!\!\right]$$

     which requires solving the $n$ equations $AA_{\cdot j}^{-1} = e_{j}$ for $j = 1, \cdots, n$ for $A_{\cdot j}^{-1}$ where $A_{\cdot j}^{-1}$ is the $j^{th}$ column of $A^{-1}$ and $e_{j}$ is the ***standard basis vector*** with all elements equal to $0$ except the $j^{th}$ element which is equal to $1$.

2. Inversion computation is actually very often prone to numerical inaccuracy 

  - *as is seen in this example taken from Keith Knight's STA410 [notes7.pdf](https://q.utoronto.ca/courses/296804/files?preview=24300633) document*
  
   $$A = \left[\begin{array}{cc}1&1-\epsilon\\1+\epsilon&1\end{array}\right] \quad \text{with analytical inverse} \quad 
A^{-1} = \left[\begin{array}{cc}\epsilon^{-2}&\epsilon^{-1}-\epsilon^{-2}\\-\epsilon^{-1}-\epsilon^{-2}&\epsilon^{-2}\end{array}\right]$$
  - and $\det(A) = |A| = A_{11}A_{22} - A_{12}A_{21} = \epsilon^{2} \not = 0 $ so $1/\det(A) = \det(A^{-1}) \not = 0$ so $A$ is mathematically ***invertible***
   

  - but if the magnitude of $\epsilon^{-2}$ outranges that of $\epsilon^{-1}$, the $\epsilon^{-1}$ terms are lost due to ***roundoff error*** and so $A^{-1}$ can never be accurately represented since in that case 

  $$A^{-1} \approx [A^{-1}]_c = \left[ \left[\begin{array}{cc}\epsilon^{-2}&\epsilon^{-1}-\epsilon^{-2}\\-\epsilon^{-1}-\epsilon^{-2}&\epsilon^{-2}\end{array}\right] \right]_c = \left[\begin{array}{cc}\epsilon^{-2}&-\epsilon^{-2}\\-\epsilon^{-2}&\epsilon^{-2}\end{array}\right]$$

  so $\det([A]_c)=0$ so $[A]_c$ is no longer ***invertible***.

In [91]:
# https://stackoverflow.com/questions/2891790/how-to-pretty-print-a-numpy-array-without-scientific-notation-and-with-given-pre
np.set_printoptions(precision=16)

# For the matrix
epsilon =  .5/100 #  2**-30 # which is not that extreme, e.g., 2**1023 # 
A = np.array([[1, 1-epsilon], 
              [1+epsilon, 1]])
# (some other potentially helpful matrix functionality:
#  e.g, np.ones, np.diag_indices, np.fill_diagonal, etc.)
print("A")
print(A)

print("Condition(A)")
print(np.linalg.cond(A))

# The analytical inverse is
A_inv = np.array([[epsilon**-2, 1/epsilon-epsilon**-2],
                  [-1/epsilon-epsilon**-2, epsilon**-2]])
print("\n\nA**-1")
print(A_inv)

# Which can be confirmed
print("\n\nA @ A_inv")
print(A @ A_inv) # matrix multiplication
print("\n\nI")
print(np.eye(2)) # identity matrix
print("\n\n(A @ A_inv) == I")
print(A @ A_inv == np.eye(2)) # Confirmation

# However, this breaks because 
# (0) general roundoff error; but, even for numbers that exactly representable  
# (1) the magnitude of epsilon**-2 will outrange epsilon**-1 if epsilon is small...

<a name="cell-sovling-SMWoodbury"></a>

### Sherman-Morrison-Woodbury Formula ([Return to TOC](#cell-solving))

---

- *The presentation in this section is taken from Keith Knight's STA410 [notes7.pdf](https://q.utoronto.ca/courses/296804/files?preview=24300633) document*. 

Also known as the ***Woodbury Matrix Identity***,

$$(A + UCV)^{−1} = A^{−1} − A^{−1}U (C^{−1} + VA^{−1}U)^{−1}VA^{−1}$$

makes inversion simple if $A$ and $C$ are diagonal.

Thus, ***low rank*** $m<n$ matrix approximations, with $A=I$ and $C=1$ 

$$
\begin{align*}
\Sigma_{n \times n}^{-1} \approx {} & (I_{n \times n} + \mathbf{u}_{n\times m}(\mathbf{v}^T)_{n\times m})^{-1}\\
= {} & I - \mathbf{u}(1+\mathbf{v}^T\mathbf{u})^{-1}\mathbf{v}^T\\
= {} & I - \frac{\mathbf{u}\mathbf{v}^T}{1+\mathbf{v}^T\mathbf{u}} \quad \text{ if } m=1
\end{align*}$$

can be used to trivialize matrix inversion approximation calculations.

In fact, performing computations on the basis of this identity can even avoid numeric problems.  Returning to the example of the previous section 

$$A = \left[\begin{array}{cc}1 & 1 - \epsilon\\ 1
+ \epsilon & 1 \end{array}\right] = \left[\begin{array}{cc}0 & - \epsilon\\  \epsilon & 0 \end{array}\right] + \left[\begin{array}{c}1 \\1 \end{array}\right]
\left[\begin{array}{c}1 \\1 \end{array}\right]^T$$

and $x=A^{-1}b$ is not a computation that will work to solve $Ax = b$ if $A^{-1}$ cannot be accurately computed; however, by instead computing

$$
x = A^{-1}b = \left(\left[\begin{array}{cc}0 & - \frac{1}{\epsilon}\\  \frac{1}{\epsilon} & 0 \end{array}\right] - 
\frac{
\left[\begin{array}{cc}0 & - \frac{1}{\epsilon}\\  \frac{1}{\epsilon} & 0 \end{array}\right] \left[\begin{array}{c}1 \\1 \end{array}\right] \left[\begin{array}{c}1 \\1 \end{array}\right]^T \left[\begin{array}{cc}0 & - \frac{1}{\epsilon}\\  \frac{1}{\epsilon} & 0 \end{array}\right]  
}{1 +  \left[\begin{array}{c}1 \\1 \end{array}\right]^T   
\left[\begin{array}{cc}0 & - \frac{1}{\epsilon}\\  \frac{1}{\epsilon} & 0 \end{array}\right]
\left[\begin{array}{c}1 \\1 \end{array}\right]}\right) b$$

$Ax = b$ can be accurately solved.

In [102]:
ep = 1e-2 #5, 7, 8,9,12
A = np.array([[1,1-ep],[1+ep,1]])
A_inv = np.array([[ep**-2, 1/ep - ep**-2],[-1/ep + -ep**-2,  ep**-2]])
b = np.array([[1],[1]])
print("epsilon", ep)
print("Condition number", np.linalg.cond(A))
print("\nA^-1 @ b = ?")
print("@ means matrix multiply")

print("\nTrue Answer")
print(np.array([[1/ep],[-1/ep]])) 
print("\nAnalytical Inverse")
print(A_inv@b)
B = np.array([[0,-ep],[ep,0]])
u = np.ones((2,1))
v = u.T
B_inv = np.linalg.inv(B)
print("\nWoodbury's Identity")
print((B_inv - (B_inv @ u @ v @ B_inv) / (1 + v @ B_inv @ u) ) @ b)
print("\nCalcluated Inverse")
print(np.linalg.inv(A) @ b)
print("\nLinear Equation Solver")
print(np.linalg.solve(A, b))
print("\nCalcluated Genearlized Inverse")
print(np.linalg.pinv(A) @ b)

epsilon 0.01
Condition number 40001.999974991086

A^-1 @ b = ?
@ means matrix multiply

True Answer
[[ 100.]
 [-100.]]

Analytical Inverse
[[ 100.]
 [-100.]]

Woodbury's Identity
[[ 100.]
 [-100.]]

Calcluated Inverse
[[ 100.]
 [-100.]]

Calcluated Genearlized Inverse
[[ 99.99999999997635]
 [-99.99999999997635]]

Linear Equation Solver
[[ 100.]
 [-100.]]


<a name="cell-sovling-condition-derivation"></a>

### [OMITTED] Condition of $x=f_{A}(b)=A^{-1}b$ ([Return to TOC](#cell-solving))

---

When computing a solultion $x=f_{A}(b)=A^{-1}b$ to $Ax = b$ for an invertible linear transformation (***square full rank***) $A$, either

$$\underset{\Large \text{$f_A$ is well-conditioned}}{b+\epsilon_b \approx b \Longrightarrow f_A(b+\epsilon_b) \approx f_A(b)} \quad \text{ or } \quad \underset{\Large \text{$f_A$ is ill-conditioned}}{b+\epsilon_b \approx b \cancel{\Longrightarrow} f_A(b+\epsilon_b) \approx f_A(b)}$$

Since actual value $\tilde x = [A^{-1}b]_c$ obtainable in the ${\rm I\!F}$ ***floating-point*** representation of ${\rm I\!R}$ will actually be a solution to a different problem 

$$A \tilde x = \tilde b \quad \text{with} \quad \tilde x = x + \epsilon_x \; \text{and}\; \tilde b = b + \epsilon_b.$$

then, for any reasonable ***vector norm*** $||\cdot||$ measuring magnitude, $A$ is called ***well-conditioned*** if 

- whenever $\frac{||\epsilon_b||}{||b||}$ is small $\quad\quad\quad\quad\quad\quad\;\;$  whenever the "nearby problem" actually being solved is "close"
- $\Longrightarrow$ then $\frac{||\epsilon_x||}{||x||}$ is also small $\quad\quad\quad\quad\quad\;\Longrightarrow$ the solution is also "close"


So a matrix $A$ is ***well-conditioned*** with respect to $x = f_A(b) = A^{-1}b$ if small changes in the input $b$ to the function, do not produces large changes in the output $x$.  Contrarily, $A$ is ***ill-conditioned*** if $f_A(b) = A^{-1}b$ is highly volatile relative to small changes in the input $b$.

The ***condition*** of $A$ in $f_A(b) = A^{-1}b$ can actually be given a precise numeric quantification on the basis of the ***induced matrix norm*** $||A|| = \underset{x\not=0}{\max} \frac{||Ax||}{||x||}$ since

\begin{align*}
\text{it implies } && ||b|| = {} &  ||Ax|| \leq ||A|| \; ||x|| \\
\text{and thus } && \frac{1}{||x||} \leq {} & \frac{||A||}{||b||} \\ \\
\text{and since } && \epsilon_x = {} & A^{-1} \epsilon_b \quad \text{ (by the definition of $\epsilon_x$ and $\epsilon_b$)},\\
&&||\epsilon_x|| = {} & ||A^{-1} \epsilon_b|| \leq {} ||A^{-1}|| \; ||\epsilon_b||\\\\
\text{the product } &&\frac{||\epsilon_x||}{||x||} \leq {} & \underbrace{||A||\;||A^{-1}||}_{\Large \kappa(A)}\;\frac{||\epsilon_b||}{||b||} \quad \text{follows}\\ 
\end{align*}

Thus the ***condition number*** of $A$ for the problem of solving for $x$ in $Ax=b$ is defined as 

|$\Large \kappa(A) = ||A||\;||A^{-1}||\quad $ | [$\Large\quad  1 \leq \kappa(A) < \infty$](https://math.stackexchange.com/questions/4116544/any-example-of-condition-number-of-matrix-less-than-1)|
|-|-|
|||

and explicitly bounds how small $\frac{||\epsilon_x||}{||x||}$ will be relative to $\frac{||\epsilon_b||}{||b||}$, characterizing how rapidly the output of the function $x = f(b) = A^{-1}b$ can change for small changes in the input $b$. Large $\kappa(A)$ means $\frac{||\epsilon_x||}{||x||}$ may not be small even if $\frac{||\epsilon_b||}{||b||}$ is small, so $A$ is ***well-conditioned*** if $\kappa(A)$ is small and ***ill-conditioned*** otherwise. 



> For the $L_2$ ***induced matrix norm***, and a ***symmetric real valued*** matrix $A$ (which thus has ***real eigenvalues***)
> 
> \begin{align*}
||A||_2 = {} & \underset{x\not=0}{\max} \frac{||Ax||_2}{||x||_2}
= \underset{||x||_2=1}{\max} \frac{||A(cx)||_2}{||(cx)||_2} 
= \underset{||x||_2=1}{\max} \frac{|c|||Ax||_2}{|c|||x||_2} 
= \underset{||x||_2=1}{\max} ||Ax||_2 \\
= {} & \underset{i}{\max} |\lambda_i| \quad \text{ (the largest magnitude eigenvalue for square $A$)}\\
\color{gray}{=} {} & \color{gray}{\sqrt{\rho(A^TA)} \quad \text{ (the square root of the spectral radius of the gramian of non-square $A$)}} 
\end{align*}
>
> and for ***symmetric*** and ***diagonizable*** ([***normal***](https://en.wikipedia.org/wiki/Normal_matrix)) $A$ with a real-valued ***orthogonal eigendecomposition*** 
>
> $$\begin{align*}
||A^{-1}||_2  = {} & ||\overset{\text{eigendecom-}}{\overbrace{(V\Lambda V^T)}^{\text{position of } A}} {}^{-1}||_2  = ||V\Lambda^{-1}V^T||_2  \\
 = {} & \frac{1}{|\lambda_\min^A|} \quad \text{ (the reciprocal of the smallest magnitude eigenvalues of $A$)}\end{align*}$$
> Thus, for the $L_2$ ***induced matrix norm***, the ***condition number*** $\kappa(A) = ||A||_2\;||A^{-1}||_2$ depends on the relative magnitudes of the smallest and largest ***eigenvalues*** with
> $$\underset{\text{the ratio of the largest and smallest eigenvalues}}{\kappa(A) = ||A||_2 ||A^{-1}||_2 = \frac{|\lambda_\max^A|}{|\lambda_\min^A|}} \quad \text{ and } \quad \frac{||\epsilon_x||_2}{||x||_2} \leq \frac{|\lambda_\max^A|}{|\lambda_\min^A|}\frac{||\epsilon_b||_2}{||b||_2}$$


<!--
> Thus for ***square full rank*** $A$, the previously noted bound on the ***spectral radius***
>
> $$\rho(A_{n\times n}) = \max_i |\lambda_i| = ||A||_2 \leq \begin{array}{c} \max_i \sum_j |A_{ij}| \\ \max_j \sum_i |A_{ij}|\end{array}$$
> 
> follows since for ***eigenvalue*** and ***eigenvector*** pair $\lambda_i$ and $v_i$, $|\lambda_i| \leq ||A||_p$ as seen from 
$$|\lambda_i| = |\lambda_i| \,||v_i||_p = ||\lambda_i v_i||_p = \overbrace{||A v_i||_p \leq ||A||_p ||v_i||_p}^{\text{by definition of the induced norm}} = ||A||_p \cancel{||v_i||_p}^1$$
>
> and the bounds are the ***Manhattan*** and ***Chebyshev distance*** ***induced matrix norms*** (as shown in Keith Knight's STA410 [class notes](https://q.utoronto.ca/courses/244990/files?preview=18669503))
>
> $$||A||_1=\max_j \sum_i |A_{ij}| \quad \text{ and } \quad ||A||_\infty =\max_i \sum_j |A_{ij}|$$
-->

<a name="cell-sovling-condition-norms"></a>

### [OMITTED] Vector and Matrix Norms ([Return to TOC](#cell-solving))

---

For $f(x) = Ax$ the statement $f(x) \approx f(x+\epsilon_x)$ means that $y = f(x) - f(x+\epsilon_x) \approx 0$ which is judged on the basis of the ***norm*** (or magnitude, or size) of the ***vector*** $y$, notated as $||y||$. The most common ***vector norms*** are the ubiquetous $L_p$ ***norms***

$$||x||_p = \left(\sum_i |x_i|^{p}\right)^{\frac{1}{p}}$$

$$||x||_2 = \underset{L_2 \text{: Euclidean}}{\sqrt{\sum_i x_i^2}} \quad   \quad ||x||_1 = \underset{L_1 \text{: Manhattan}}{\sum_i |x_i|} \quad  \quad ||x||_\infty = \underset{L_\infty \text{: Chebyshev}}{\max_i |x_i|}$$

- ***Norms*** will be considered again in the context of function (vector) spaces. Some additional considerations regarding ***norms*** are available in Keith Knight's STA410 [class notes](https://q.utoronto.ca/courses/244990/files?preview=18669503).

We can also define the ***norm*** (or magnitude, or size) of a matrix $A$.  One common method to do so is to induce a ***matrix norm*** from the $L_p$ ***vector norms*** as 

$$||A||_p = \underset{x\not=0}{\max} \frac{||Ax||_p}{||x||_p}$$

Another common ***matrix norm*** is the ***Frobenius norm***
$$||A||_F = \left(\sum_i\sum_j A_{ij}^2\right)^{\frac{1}{2}} =  \underbrace{\sqrt{\text{tr}(AA^T)} = \sqrt{\text{tr}(A^TA)} }_{\text{trace doesn't care about transposes}} = \underset{\text{singular values of } A}{\overset{\text{The $L_2$ norm of the}}{\sqrt{ \sum_i \lambda_i^2}}}$$

which is just a direct extension of $L_2$ ***Euclidean distance***. 

- Further analysis of ***condition*** in the context of $L_p$ ***induced matrix norms*** is available in Keith Knight's STA410 [class notes](https://q.utoronto.ca/courses/244990/files?preview=18669503); and, $L_p$ ***induced matrix norms*** themselves are also more fully analyzed in Keith Knight's STA410 [class notes](https://q.utoronto.ca/courses/244990/files?preview=18669503), e.g., using ***Manhattan*** and ***Chebyshev distance*** ***induced matrix norms*** to derive (the previously provided) bounds on the ***spectral radius***.


<a name="cell-sovling-inverses"></a>

### [OMITTED] Generalized Inverses ([Return to TOC](#cell-solving))

---

Returning to our system of linear equations $Ax = b$, it is very straightforward to analyticall calculate $x = A^{-1} b$ for ***full rank square*** matrices $A$ if we have the ***eigendecomposition*** or ***SVD*** of $A$ since 

- if $A$ is a ***symmetric*** and ***full rank*** then $A = V \Lambda V^T$ and $A^{-1} = V \Lambda^{-1} V^T$ with $\Lambda^{-1}_{ii} = \frac{1}{\Lambda_{ii}}$ since

\begin{align*}
A^{-1}A = {} & V \Lambda^{-1} V^T V \Lambda V^T\\
= {} & V \Lambda^{-1} \;\;\, I \;\;\, \Lambda V^T\\
= {} & V \quad \;\;\, I \;\;\, \quad V^T = I\\
\end{align*}

- if $A$ is ***square*** and ***full rank*** $A = U D V^T$ and $A^{-1} = V D^{-1} U^T$ with $D^{-1}_{ii} = \frac{1}{D_{ii}}$ since

\begin{align*}
A^{-1}A = {} & V^T D^{-1} U U^T D V\\
= {} & V^T D^{-1} \;\;\, I \;\;\, D V\\
= {} & V^T \quad \;\;\; I \;\;\;\quad V = I\\
\end{align*}

Further, $Ax = b$ can be ***consistent*** (i.e., have at least one solution) even if $A$ is not squre or full rank.

- $Ax = b$ is ***consistent*** if 

  $$\text{rank}(A|b) = \text{rank}(A)$$

  where $A|b$ is the matrix made by appending the column $b$ as the rightmost column of $A$.

If $Ax = b$ is ***consistent***, then a solution $x = A^{-}b$ based on $A^{-}$ only requires that 

$$A = A A^{-}A $$

since

\begin{align*}
 Ax = {} & A A^{-}Ax\\
  = {} & A A^{-}b\\
 \Longrightarrow x = {} & A^- b \quad \text{ is a possible solution}\\
\end{align*}

Matrices which can play the role of $A^{-}$ above, from strongest to weakest, are

0. $A^{-1}$:  ***inverses*** which satisfy $A^{-1}A = AA^{-1} = I$
1. $A^{+}$: (unique) ***Moore-Penrose inverses*** for which $A^{+}A$ and $AA^{+}$ are ***symmetric*** and which are also ***g1*** and ***g2 inverses***
  > also called ***p-inverses***, ***normalized generalized inverses***, or ***pseudoinverses***
2. $A^{*}$: ***g2 inverses*** for which $A^{*}AA^{*} = A^{*}$ and which are also ***g1 inverses***
  > also called ***reflexive generalized inverses*** and ***outer pseudoinverses***
3. $A^{-}$: ***g1 inverses*** for which $A A^{-}A = A$
  > also called ***conditional inverses*** or ***inner pseudoinverses***

Now, if $Ax = b$ is ***consistent***, then 

$$x = A^{+}b \quad \text{is a solution to} \quad Ax = b$$

where 
- $A^{+} = V D^{+} U^T$ and $D^{+}_{ii}=\frac{1}{D_{ii}}$ is taken from the SVD $A = U D V^T$ 
and can be seen to satisfy $AA^{+}A = A$ and be the (unique) ***Moore-Penrose inverse***.

<a name="cell-sovling-notAxbwAinv"></a>

## 3.3.1 Not $A^{-1}$ ([Return to TOC](#cell-solving))

---

The `np.linalg.solve(A,b)` approach is more computationally efficient than `np.linalg.inv(A)@b`. So how does `np.linalg.solve(A,b)` work?

<a name="cell-sovling-backsub"></a>

### [PREREQUISITE] Backward Substitution ([Return to TOC](#cell-solving))

---

Consider the easier problem of solving for $x$ in $A_{n \times n}x = b$ when $A_{n \times n}$ is given in ***upper triangular form***, where everything below the diagonal is zero and everything on the diagonal is non-zero.

$$\left[\begin{array}{cccccc} 
a_{11}&a_{12}&a_{13}& \cdots & a_{1(n-1)} & a_{1n}\\
 &a_{22} &a_{23} & \cdots &a_{2(n-1)} & a_{2n} \\ 
 &&a_{33} & \cdots &a_{3(n-1)} & a_{3n} \\ 
 &&& \ddots & \vdots & \vdots \\
& &&& a_{(n-1)(n-1)}& a_{(n-1)n}\\
0 & &&& & a_{nn}\\
\end{array}\right] 
\left[\begin{array}{c} 
x_1\\x_2\\x_3\\\vdots\\x_{n-1}\\x_{n}\\
\end{array}\right] = 
\left[\begin{array}{c} 
b_1\\b_2\\b_3\\\vdots\\b_{n-1}\\b_{n}\\
\end{array}\right]$$

In this form $x$ can be solved for using ***backward substitution*** as

$$x_n = \frac{b_n}{a_{nn}} \quad x_{n-1} = \frac{b_{n-1} - a_{(n-1)n}x_n}{a_{(n-1)(n-1)}} \quad \cdots \quad x_{n-j} = \frac{b_{n-j} - \sum_{i=n}^{n-j+1}a_{(n-j)i}x_i}{a_{(n-j)(n-j)}}$$

so long as (the so-called ***pivot points***) $a_{jj} \neq 0$ so there is no division by zero. 

For $x_j$, the final formula shows that there is $1$ division and $n-j$ multiplications and $n-j$ subtractions, so the total number of arithmetic computations to solve for all $x_j$ is 

$$\sum_{j=n}^1 1 + 2(n-j) = \sum_{j=0}^{n-1} (1 + 2j) = n + 2 \sum_{j=0}^{n-1} j = n + 2\frac{n(n-1)}{2} = n^2$$

> The presentation above is given for ***square invertible*** A; but, the ***backward substitution*** algorithm can also find solutions to ***non-square*** systems of equations based on $A_{n\times m}$ were the ***upper triangular form*** is exchanged with [row echelon form](https://en.wikipedia.org/wiki/Row_echelon_form) (where every row must have more leading zeros than the row above it). A system $A_{n\times m}x = b$ itself may be 
> 
> - ***overdetermined*** ($n>m$) with more equations (rows) than the unknown variables (and the "triangle" completes before the final row of $A$)
> - ***underdetermined*** $(n<m)$ so there are more free unknown variables than the number of equations (and the triangle doesn't complete before before the final row of $A$)
> 
> and the system $A_{n\times m}x = b$ may be 
> 
> - ***consistent*** *with a single solution*, e.g.,    
>
>   $$\left[\begin{array}{cc}
1 & 1\\
0 & 1
\end{array}\right]
\left[\begin{array}{c}
x_1\\
x_2
\end{array}\right] = 
\left[\begin{array}{c}
1\\
1
\end{array}\right] 
$$
> 
>   in which case $\text{rank}(A) = \text{rank}(A|b)$, and the columns (and rows) of $A$ are ***linearly independent*** so no columns (and rows) will be linear combinations of each other
> 
> - ***consistent*** *with infinitely many solutions*, e.g., 
>
>   $$\left[\begin{array}{cc}
1 & 1\\
0 & 0
\end{array}\right]
\left[\begin{array}{c}
x_1\\
x_2
\end{array}\right] = 
\left[\begin{array}{c}
1\\
0
\end{array}\right] 
$$
> 
>   in which case $\text{rank}(A) = \text{rank}(A|b)$, but some columns (and rows) of $A$ are ***linearly dependent*** so some columns (and rows) will be linear cominations of each other
>   
>   > i.e., for some sets of indices $\mathcal{J} = \{j_k: k=1,...,K\}$ and $\mathcal{I} = \{i_k: k=1,...,K\}$
>   >
>   > $$\sum_{j \in \mathcal{J}} c_j A_{*j} = 0  \;\; \not \! \Longrightarrow  \;\; c_j = 0 \quad \text{ and } \quad \sum_{i \in \mathcal{I}} c_i A_{i*} = 0  \;\; \not \! \Longrightarrow  \;\; c_i = 0$$
> 
> - ***inconsistent*** *with no solutions at all*, e.g., 
> 
>   $$\left[\begin{array}{cc}
1 & 1\\
0 & 0
\end{array}\right]
\left[\begin{array}{c}
x_1\\
x_2
\end{array}\right] = 
\left[\begin{array}{c}
0\\
1
\end{array}\right] 
$$
> 
>   in which case $\text{rank}(A) < \text{rank}(A|b)$, and the column $b$ cannot be constructed as a linear combination of the columns of $A$.
> 
> where $A|b$ is the $n \times (m+1)$ [*augmented matrix*](https://en.wikipedia.org/wiki/Augmented_matrix)
> 
> $$\left[\begin{array}{ccc:c} 
a_{11} & \cdots & a_{1m} & b_1 \\
\vdots & \ddots & \vdots  & \vdots \\
a_{n1} & \cdots & a_{nm} & b_m \end{array}\right]$$





<a name="cell-sovling-elimination"></a>

## [PREREQUISITE] Gaussian Elimination ([Return to TOC](#cell-solving))

---

Converting a system of linear equations into ***upper triangular form*** (or the more general ***row echelon form***) is itself a quite simple process known as ***Gaussian elimination***.

- Multiplying a row of the augmented matrix $A|b$ by a constant and adding to another row of the augmented matrix produces an equivalent system of linear equations to the one originally defined by the augmented matrix, i.e.,

  $$x \quad \text{ solving } \quad E^{ci+j}Ax = E^{ci+j}b \quad \text{ also solves } \quad Ax = b$$

- Multiplying and adding rows in this manner can produce leading zeros, e.g.,

  $$\left[\begin{array}{ccc:c} 
a_{11} & \cdots & a_{1m} & b_1 \\
\vdots & \ddots & \vdots  & \vdots \\
a_{n1} & \cdots & a_{nm} & b_n \end{array}\right]
\quad \overset{A|b \; \rightarrow \; E^{c1+m}[A|b]}{\longrightarrow} \quad
\left[\begin{array}{ccc:c} 
a_{11} & \cdots & a_{1m} & b_1 \\
\vdots & \ddots & \vdots  & \vdots \\
a_{n1} + ca_{11} & \cdots & a_{nm} +c a_{1m} & b_n + cb_1\end{array}\right]$$

  and if $c = -\frac{a_{n1}}{a_{11}}$ then $a_{n1} + ca_{11} = 0$ and the bottom left element of the resulting matrix vanishes (i.e., becomes $0$).

When the column elements below a ***pivot point*** have been turned into zeros, the column and row of the ***pivot point*** are completed, the the ***Gaussian elimination*** process recurrsively restarts on the next ***pivot points*** in the top left corner of the submatrix without the completed row and column. 

$$\left[\begin{array}{c|ccc:c} 
a_{11} & a_{12} &  \cdots & a_{1m} & b_1 \\\hline
0 & a_{22} + c_2 a_{2m} & \cdots & a_{2m} + c_2 a_{1m} & b_2 + c_n b_1 \\
\vdots & \vdots & \ddots & \vdots  & \vdots \\
0 & a_{n2} + c_n a_{n1} & \cdots & a_{n2} + c_n a_{nm} & b_n + c_n b_1\end{array}\right]$$

For a ***square*** matrix $A$ where $m=n$, the above formulation [shows](http://www.it.uom.gr/teaching/linearalgebra/chapt6.pdf) that the number of divisions and multiplication-additions that are required to create an ***upper triangular form*** matrix (augmented with the transformed $b$ column) are

$$\sum_{j=1}^n (j+1)(j-1) + \underset{\text{due to } b}{(j-1)} = \sum_{j=1}^n j^2 - 1 + (j-1) = \frac{n(n+1)(2n+1)}{6} - n + \frac{n(n+1)}{2} - n $$

> #### Pivoting
> An important computational caveat is that when the scalar multiplier $c$ is large, the numerical precision of the floating point-operation will be insufficient if 
> $$[b_{i'} + cb_i]_c = [cb_i]_c$$ 
> e.g., for three digits of precision, one step of ***Gaussian elimination*** on 
>
> \begin{align*}
0.0001 x_1 + x_2 & {} = 1\\
x_1 + x_2 & {} =  2\\
\quad \quad \quad \quad \quad \; \text{produces } \quad \quad \quad & {}    \\
 \quad 0.0001 x_1 + x_2 & {} = 1\\
 -10000x_2 & {} = -10000 \quad \text{ (the "$+x_2$" and the 2 are lost due to precision!)}\\
\end{align*}
>
> To fix this issue the rows may be reordered with a ***partial pivot*** so $c$ will be as small as possible, and ***Gaussian elimination*** step will then be 
>
> \begin{align*}
x_1 + x_2 & {} =  2\\
0.0001 x_1 + x_2 & {} = 1\\
\text{instead produces } \quad \quad \quad & {}    \\
x_1 + x_2 & {} =  2\\
x_2 & {} = 1 \quad \text{ (has roundoff error but solution's more accurate)}\\
\end{align*}
>
> which gives $x_1=x_2=1$ which is a more accurate solution than $x_1=0$ $x_2=1$.
> 
> If this was not sufficient, a ***full pivot*** which reorders both the rows and the columns as well could be used to produce an even smaller $c$.
>
> *This example is inspired by the **Pivoting** subsection of the 
Section 5.2 **Gaussian Elimination and Elementary Operator Matrices** in Chapter 5 **Numerical Linear Algebra** on page 212 of James E. Gentle's **Computational Statistics** textbook.*

<a name="cell-sovling-elementary"></a>

## [PREREQUISITE] Elementary Operations ([Return to TOC](#cell-solving))

---

Mathematically, multiplying row $i$ by scalar $c$ and adding it to row $j$, and ***partial pivoting*** two rows $i$ and $j$ are so-called ***elementary operations*** and are represented by simple matrix multiplications $E^{ci+j}[A|b]$ and $E^{i\leftrightarrow j}[A|b]$, respectively, where

- $E^{ci+j} = I + c e_je_i^T$, so $E^{ci+j}_{kk}=1$ and $E^{ci+j}_{ji}=c$ and all other entries of $E^{ci+j}$ are $0$.

  - $\left(E^{ci+j}\right)^{-1} = E^{-ci+j}$ since $E^{ci+j} E^{(-c)i+j} =  E^{(-c)i+j}E^{ci+j} = I$.

- $E^{i\leftrightarrow j} = I^{i\leftrightarrow j}$ where row $i$ and $j$ have in the identity matrix $I$ have been switched, so all $E^{i\leftrightarrow j}_{kk}=1$ except $E^{i\leftrightarrow j}_{ii} = E^{i\leftrightarrow j}_{jj} = 0$ and all other elements are $0$ except $E^{i\leftrightarrow j}_{ij}=E^{i\leftrightarrow j}_{ji}=1$.

  - $(E^{i\leftrightarrow j})^{-1} = E^{i\leftrightarrow j}$ since $E^{i\leftrightarrow j}E^{i\leftrightarrow j}=I$ so it's (of course) easy to "undo" row interchanges.


$$
E^{ci+j} = \left[\begin{array}{ccccccc}
1 & &&&&&0\\
&1&&&&&\\
 && \ddots&&\\
& && 1 &&\\
 &&c&& \ddots\\
&&\uparrow&&&1&\\
0&&E^{ci+j}_{ji}&&&&1\\
\end{array}\right] 
\quad\quad
E^{i\leftrightarrow j} = \left[\begin{array}{ccccccc}
1 &0&&&&0&0\\
0& \ddots &&&&&0\\
 &\cdots & 0 &\cdots& 1&\cdots \\
 &&& \ddots \\
 &\cdots & 1 &\cdots& 0 &\cdots \\
 0 &&&&& \ddots &0\\    
0  &0&&&&0& 1 \\  
\end{array}\right]
\begin{array}{c}\leftarrow \text{ row }i\\\\\leftarrow \text{ row }j\\\end{array}
$$


<a name="cell-sovling-lu"></a>

### 3.3.1.0 The LU Decomposition ([Return to TOC](#cell-solving))

---

Ignoring row interchanges $E^{i\leftrightarrow j}$ which are easily applied and undone, ***Gaussian elimination*** transformation sequence 

$$\prod E^{ci+j} \quad \text{and} \quad \left(\prod E^{ci+j}\right)^{-1} = \prod E^{-ci+j} = L$$ 

will be ***lower triangular matrices*** and 

  $$U = \left(\prod E^{ci+j}\right) A$$ 

[may](https://math.stackexchange.com/questions/218770/when-does-a-square-matrix-have-an-lu-decomposition/2274657) (if ***Gaussian elimination*** is working) be an ***upper triangular***, and when so
\begin{align*}
Ax = {} & b\\
\left(\prod E^{ic+j}\right) Ax = {} & \left(\prod E^{ic+j}\right) b\\
Ux = {} & L^{-1}b
\end{align*}

and $x$ may be solved for by simple ***backward substitution***.

***LU decomposition*** is thus seen to be a byproduct of solving for $x$ using ***Gausian elimination*** where operation is done using the extended augmated matrix [$A|I|b$](https://en.wikipedia.org/wiki/Gaussian_elimination#Finding_the_inverse_of_a_matrix) instead of only $A|b$  

$$\left[\begin{array}{ccc:ccc:c} 
a_{11} & \cdots & a_{1m} & 1 & \cdots & 0 & b_1 \\
\vdots & \ddots & \vdots  & \vdots & \ddots & \vdots & \vdots  \\
a_{n1} & \cdots & a_{nm} & 0 & \cdots & 1 & b_n \end{array}\right]
\quad \overset{[A|I|b] \;\rightarrow \;L^{-1}[A|I|b]}{\longrightarrow} \quad 
\left[ \!\begin{array}{c:c:c}  U & L^{-1} & b' \!\end{array}  \right]$$

Computationally all that needs to be kept track of is the sequence of the ***elementary operations*** $\left(\prod E^{ic+j}\right)$ actualized during the ***Gausian elimination*** process, since 

$$U = \left(\prod E^{ic+j}\right) A \quad \text{and} \quad
\left(\prod E^{ic+j}\right)I = L^{-1} \quad \text{and} \quad
\left(\prod E^{ic+j}\right)b = b'$$

> The ***LU decomposition*** will be [unique](https://math.stackexchange.com/questions/1799854/is-the-l-in-lu-factorization-unique) if
>
> $$x^TAx \geq 0 \quad\quad \text{ subject to the constraint } \quad \quad L_{kk} = 1 \; \text{ or } \; U_{kk} = 1 \; \text{ for all $k$}$$
>
> i.e., if $A$ is a ***square nonnegative definite matrix***, and the diagonals of either $L$ or $U$ all one.

<a name="cell-sovling-lulike"></a>

## 3.3.2 Finding $X\hat \beta \approx y$ by solving for $\hat \beta$ in $\overset{A\, x}{X^T\!X \hat \beta} = \overset{b}{X^t y}$ ([Return to TOC](#cell-solving))

---

The linear regression model $y = X\beta + \epsilon$ can be re-expressed as the ($n>m$) ***overdetermined*** problem

$$\begin{align*}
 \overbrace{X_{n \times m} \hat \beta}^{A\,x} = {} & \overbrace{y - \hat \epsilon}^{b} 
\end{align*}$$

Of course unless $\epsilon=0$, this leads to the ***inconsistent*** system of equations $X \hat \beta \not = y$. However, $X \hat \beta \approx y$ if 

<!--
$$\begin{align*}
\hat \beta = {} & \underset{\quad \; \beta: \; \hat \epsilon \; = \; y - X\beta }{\text{argmin } ||\hat \epsilon||_2^2} \\
 = {} & \underset{\beta}{\text{argmin }} ||y - X \beta ||_2^2 = \underset{\beta}{\text{argmin }} (y - X \beta )^T(y - X \beta ) \\
\end{align*}
$$
-->

\begin{align*}
  \hat \beta = {} & \underset{\quad \; \beta: \; \hat \epsilon \; = \; X\beta - y}{\text{argmin } ||\hat \epsilon||_2^2} 
  = \underset{\beta}{\text{argmin }} ||y-X \beta||_2^2  \\
  = {} & \underset{\beta}{\text{argmin }} (y-X \beta)^T(y- X \beta) 
  = \underset{\beta}{\text{argmin }} \underbrace{\beta^TX^TX \beta -2\beta^TX^Ty}_{f(\beta)} \\
  0 = {} & \overbrace{\nabla_\beta \left( \beta^TX^TX \beta -2\beta^TX^Ty \right)}^{\nabla_\beta f(\hat \beta )}\\
  0 = {} &  X^TX \hat \beta - X^T y  \quad \Longrightarrow \quad  \underset{{\text{$\sum_i \hat \epsilon_i = 0$ if $X$ has an intercept column} }}{\overset{\text{residuals are othogonal to columns $X_{*j}$}}{0 = X^T (X \hat \beta - y) = X^T\hat \epsilon }} \\
  \Longrightarrow \quad X^T y = {} &  X^TX \hat \beta \quad \longleftarrow \quad \textrm{the so-called } \textit{normal equations}
  \end{align*}

so that $\hat \beta$ above is the solution to the ***normal equations*** $\overbrace{(X^TX) \hat \beta}^{A\,x} = \overbrace{X^TY}^{b}$, which leads to the familiar analytical solution $\underbrace{\hat \beta}_x = \underbrace{(X^TX)^{-1}}_{A^{-1}}\underbrace{X^TY}_b$.

So 

$$X \hat \beta\approx y \quad \text{ transforms into } \quad X^TX\hat \beta = A^Tb \quad (\text{or generically } Ax = b)$$ 

such that solving for $\hat \beta$ in the latter provides $X \hat \beta\approx y$.

<a name="cell-sovling-gradients"></a>

### [PREREQUESITE] Gradients ([Return to TOC](#cell-solving))

--- 

The ***operator*** (i.e., function returning a function) $\nabla_{x}$ is called the ***gradient*** and generalizes the ***derivative for scalar quantities*** as the vector collection of all the ***partial derivatives*** of its argument, the ***scalar valued multivariate function*** $f_x$, with respect to input elements $x_i$ of $f_x$

$$ \nabla_{x}(f_x) = \frac{\partial f_x}{\partial x_1} e_1 + \cdots + \frac{\partial f_x}{\partial x_i} e_i + \cdots + \frac{\partial f_x}{\partial x_n} e_n $$

for ***standard basis vectors*** $e_i$.

Like the ***derivative***, the ***gradient*** is a ***linear operator*** with 
 
$$\nabla_{x}(af_x+bg_x) = a\nabla_{x}(f_x) + b\nabla_{x}(g_x)$$

and (as seen above) has the simple rules

$$\nabla_{x} x^Tb = b \quad \text{ and } \quad \nabla_{x} x^TAx = 2Ax$$

<a name="cell-sovling-qr"></a>

### 3.3.2.0 The $QR$ Decomposition ([Return to TOC](#cell-solving))

---

The ***(modified) Gram-Schmidt procedure*** applied to the design matrix $X$ will create a ***semi-orthonormal*** $Q$. Since in the ***(modified) Gram-Schmidt procedure*** column $j$ of $Q$ is created using the first $j$ columns of $X$, it must be the case then that 

$$X = QR$$

for some ***upper trapezoidal matrix*** $R$ so that column $j$ of $X$ is also created using first $j$ columns of $U$. This ***QR decomposition*** is ***rank revealing*** when $X$ is not ***full rank***, since if any columns of $X$ are ***linearly dependent*** the ***(modified) Gram-Schmidt procedure*** will produce $\mathbf{0}$ column vectors

$$\overset{\text{rank } r}{X_{n\times m}} = 
\overset{\text{orthonormal}}{\left[\overbrace{Q_{n\times r}}^{\text{semi-}} \; \Big | \; \mathbf{0}_{n\times (n-r)}\right]}_{n \times n} \overset{\text{trapezoidal}}{\left[\begin{array}{c}R_{r\times m}\\\mathbf{0}\end{array} \right]}_{n \times m} = \;\;\;Q_{n\times r}R_{r\times m}$$

with ***semi-orthonormal*** $Q$ so that

$${Q_{\cdot j}^TQ_{\cdot j'} = \left\{\begin{array}{l}1: \text{if } j=j' \quad  \text{i.e., the columns are normal vectors}\\0: \text{if } j\not=j'  \quad \text{i.e., the columns are orthogonal}\end{array}\right.}$$

> As noted beginning on page 39 of Keith Knight's STA410 [notes9.pdf](https://q.utoronto.ca/courses/244990/files/18669511) document, even though the ***(modified) Gram-Schmidt procedure*** is more numerically precise that the original ***Gram-Schmidt procedure***, the $Q$ matrix can be even more precisely calculated using ***Householder transformations*** (or ***reflections***); or, using ***Givens transformations*** (or ***rotations***), which are especially efficient if the data arives from memory row by row. 

When $X$ is ***full rank***, $R$ will be an ***upper triangular (full rank) matrix***, so that the normal equations

\begin{eqnarray*}
X^TX \hat \beta = & X^T y \\
\Longrightarrow R^TR \hat \beta = & R^TQ^T y\\
\overset{(R^T)^{-1} \text{ exists}}{\quad \quad \Longrightarrow} R \hat \beta = & Q^T y
\end{eqnarray*}

and lead to the [general advice](https://www.quora.com/Is-it-better-to-do-QR-Cholesky-or-SVD-for-solving-least-squares-estimate-and-why) that computing the ***QR decomposition*** and then solving for $\hat \beta$ using ***backward substitution*** with $R$ is favorable to solving for $\hat \beta$ based on computing $X^TX$.

<!-- 

***QR Decomposition*** can also speed up solving ($n>m$) ***overdetermined*** (but ***consistent***) $A_{n \times m}x=b$ (as opposed to $Ax\approx b$) whenever $\text{rank}(A) =r << m$ since 

$$A_{n \times m}x = \left[Q_{n \times r}|Q_{n \times (m-r)}'\right] \left[\begin{array}{c}R_{r\times m}\\\mathbf{0}\end{array} \right] x=b \quad \text{ so } \quad \left[\begin{array}{c}R\\\mathbf{0}\end{array} \right]x = \left[\begin{array}{c}Q^T\\ Q'^T\end{array} \right] b$$

and (when $A_{n \times m}x=b$ is indeed ***consistent***)

$$R_{r\times r} \overbrace{\left[\begin{array}{c}\tilde x_{r\times 1}\\\mathbf{0}\end{array} \right]}^{x} =  \overbrace{\left[\begin{array}{c} Q^T_{r \times n} b_{n\times 1}\\\mathbf{0}\end{array} \right]}^{Q^Tb} \quad \text{ so } \quad R_{r \times r} \tilde x = Q^Tb$$

and this is a very small ***backward substitution*** compared to $L_{n \times n}^{-1} A_{n \times m}x = U_{n \times m}x = L_{n \times n}^{-1}b$.

So even though $O(n^2m)$ ***QR decomposition*** will be about 5 times slower than ***Gaussian*** elimination (as seen above), this disadvantage may be made up for during the ***backward substitution*** step where $O(r^2)$ can be substantially faster than $O(nm)$.  And, actually, the general solution to an ***overdetermined*** system is to use the ***pseudoinverse*** available from ***SVD***. 

n = 1000
A, b = stats.norm.rvs(size=(n,n)), stats.norm.rvs(size=(n,1))

%timeit np.linalg.qr(A)

%timeit np.linalg.solve(A,b)

n,m = 2000,10
X, y = stats.norm.rvs(size=(n,m)), stats.norm.rvs(size=(n,1))
y = X[:,0]+X[:,1]

%timeit Q,R = scipy.linalg.qr(np.tile(X,100)); np.linalg.solve(R[:10,:10], (Q.T@y)[:10])

# np.linalg.lstsq(X,y, rcond=None)
# https://stats.stackexchange.com/questions/240573/how-does-numpy-solve-least-squares-for-underdetermined-systems
%timeit np.linalg.lstsq(np.tile(X,100),y, rcond=None)
# Actually, numpy doesn't do an LU decomposition / Guassian elimination approach
# rather, it uses SVD; still, this maintains a larger memory footprint than QR
# https://math.stackexchange.com/questions/3252351/when-solving-a-linear-system-why-svd-is-preferred-over-qr-to-make-the-solution
-->

<!-- 
- The use of the ***QR decomposition*** for estimating a least squares linear model fit will be considered further in [Programming Portfolio Assignment 1 Problem 2](#cell-solving-start). 
-->

<!-- a very interesting point to follow up on with Cholesky... could cholesky work?? can add this to the homework... -->
<!-- https://www.quora.com/Is-it-better-to-do-QR-Cholesky-or-SVD-for-solving-least-squares-estimate-and-why -->

<a name="cell-sovling-chol"></a>

### 3.3.2.1 The Cholesky Decomposition ([Return to TOC](#cell-solving))

---

If the ***condition number*** $\kappa(X^TX)$ is not too large, then computing $X^TX$ might incur minimal ***roundoff error***, and assuming $X_{n \times m}$ is ***full rank***, $(X^TX)_{m \times m}$ will be ***full rank*** and ***symmetric*** 

- which implies it has ***positive eigenvalues*** 
  - which implies it ***positive definiteness*** so $x^TX^TXx > 0$

For $\Sigma$ which is ***symmetric*** $\Sigma = \Sigma^T$ and ***positive definite*** $x^T\Sigma x > 0$, there exists a ***Cholesky decomposition*** 

$$\Sigma = CC^T \quad \text{with lower triangular }C \text{ (which is unique when $C_{ii} >0$)}$$

so that $\hat \beta$ can be solved for in 

  $$\begin{align*}
  X^T\!X\hat \beta = {} & X^T\!y\\
= CC^T \hat \beta = {} & X^T\!y \\
\Longrightarrow  \text{ by solving for $\gamma$ in } \underset{L}{C} (\underbrace{C^T \hat \beta}_{\gamma}) = {} & X^T\!y \; \text{ and then $\hat \beta$ in} \; \underset{U}{C}^T \hat \beta = \gamma\\
  \end{align*}$$

- The computation of the ***Cholesky decomposition*** will be examined further in [Week 2 Programming Assignment Problem 2]().

The ***Cholesky decomposition*** is notably advantageous relative to ***LU decomposition*** in terms of

- ***computation*** because while also being $O(n^3)$, it can be constructed with half of as many operations
- ***storage*** because since $C$ is repeated, it only requires $n(n+1)/2$ elements, half of the $n^2+n$ elements required in ***LU decomposition*** 
- ***inversion*** since $(CC^T)^{-1} = (C^T)^{-1}C^{-1} = $ [$ \;(C^{-T})C^{-1}$](https://math.stackexchange.com/questions/340233/transpose-of-inverse-vs-inverse-of-transpose) $= (C^{-1})^TC^{-1}$  only requires the computation of $C^{-1}$, while $(LU)^{-1} = U^{-1}L^{-1}$ requires both $L^{-1}$ and $U^{-1}$, the ***Cholesky decomposition*** is twice as fast.

and so the ***Cholesky decomposition*** is an interesting exception to the typical ***time-space tradeoff*** since its construction outperforms ***LU decomposition*** both in computation *and* memory requirements. For additional details and commentary on ***Cholesky decomposition***, please see Keith Knight's STA410 [notes7.pdf](https://q.utoronto.ca/courses/296804/files?preview=24300633) document beginning on page 13 and [notes9.pdf](https://q.utoronto.ca/courses/296804/files?preview=24301088) beginning on page 37.

<a name="cell-sovling-chol-mvn"></a>

### Application to Covariance Matrices ([Return to TOC](#cell-solving))

---

A ***covariance matrix*** $\Sigma$ is a ***symmetric positive definite*** matrix, and therefore can be considered in terms of its ***Cholesky decomposition*** 


$$Y \sim MVN(\mathbf{0},I) \quad \Longrightarrow \quad C Y \sim MVN(\mathbf{0}, \Sigma=CC^T)$$

A ***covariance matrix*** represented by its ***Cholesky decomposition*** therefore has reduced *storage* requirements, and has *reduced inverse computation requirements* since $\Sigma^{-1} = (CC^T)^{-1} = (C^{-1})^TC^{-1}$, which makes the ***Cholesky decomposition*** especially attractive for computational purposes in the context of ***covariance matrices***. 


> ***Cholesky decomposition*** is sometimes referred to as the ***square-root decomposition***; however, for ***symmetric nonnegative definite*** $A$ the ***square-root decomposition*** is already defined as
>
> $$A^{\frac{1}{2}} = VD^{\frac{1}{2}}V^T \quad \text{ such that } \quad \left(A^{\frac{1}{2}}\right)^2= VDV^T = A$$
>
> for ***diagonal*** $D$ and ***orthogonal*** $V$.  ***Square-root decomposition*** often proves useful in theoretical settings, whereas the computational efficiencies of ***Cholesky decomposition*** lead to its ubiquity in applied settings.

<a name="cell-sovling-svd"></a>

### 3.3.2.2 The return of the SVD ([Return to TOC](#cell-solving))

---

If $A^{-1}$ exists, then from the ***SVD*** $A = UDV^T$ and $A^{-1} = V D^{+} U^T$ where $D^{+}_{ii}=\frac{1}{D_{ii}}$.  Thus, $x = A^{-1}b$ solving $Ax = b$ can be calcuated as $x = V D^{+} U^Tb$.

> Similarly, $\hat \beta$ solving $X^TX\hat \beta = X^Ty$ and $X\hat \beta \approx  y$ can be computed from the ***SVD*** of $X = UDV^T$ which 
provides the (unique) ***Moore-Penrose (generalized) inverse*** $(X^TX)^+ = V (D^{+})^2 V^T \underset{\text{full rank}}{\overset{\text{if $X$ is}}{=}} (X^TX)^{-1}$ with $(D^{+}_{ii})^2=\frac{1}{D_{ii}^2}$ so that $\hat \beta=V (D^{+})^2 V^T X^T y$.